In [1]:
import ast
import os
from typing import Dict, Any, List, Union, Optional
import openai
import json

In [2]:
# Global dictionaries for everything:
global_functions: Dict[str, Union[ast.FunctionDef, ast.AsyncFunctionDef]] = {}
global_variables: Dict[str, ast.AST] = {}  # e.g., for top-level list/dict nodes

def build_global_registry(root_dir: str, root_package: str) -> None:
    """
    Recursively walk `root_dir` (containing your Python package),
    parse each .py file to find top-level function defs and variable defs
    (that are lists or dicts).

    We'll store them in global_functions and global_variables with a fully
    qualified name, e.g. "some_package.module.foo_func".
    """
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".py"):
                fullpath = os.path.join(dirpath, filename)
                # Infer the module name from dirpath relative to root_dir, plus filename
                module_name = derive_module_name(dirpath, filename, root_dir, root_package)
                parse_for_top_level_defs(fullpath, module_name)

def derive_module_name(dirpath: str, filename: str, root_dir: str, root_package: str) -> str:
    """
    Convert a file path into a dotted module name. For example,
    if root_dir = /path/to/my_project, root_package = "my_project",
    and the file is /path/to/my_project/some_package/sub_module.py,
    then we might return "some_package.sub_module".
    """
    # This is somewhat custom depending on your directory structure.
    # The general approach:
    # 1. remove `root_dir` from the start of dirpath
    # 2. split the remainder by os.sep
    # 3. remove any empty strings or __pycache__
    # 4. append filename without .py
    # 5. join with '.'
    rel_path = os.path.relpath(dirpath, root_dir)  # relative to the root
    parts = []
    if rel_path != ".":
        parts = rel_path.split(os.sep)

    # remove .py extension from filename
    base_name = filename[:-3]  # remove .py
    parts.append(base_name)

    # e.g. parts = ["some_package", "sub_module"]
    # combine them with '.'
    # If you want a leading "my_project" or "root_package", you can decide
    # whether to keep that or not:
    # For example, we might skip "my_project" if it's the top-level.
    # Or we might do:
    #    return ".".join([root_package] + parts)
    # depending on how you want the FQN to look.

    module_name = ".".join(parts)
    return module_name


def parse_for_top_level_defs(filepath: str, module_name: str) -> None:
    """
    Parse one file, collecting top-level function definitions and
    top-level variable definitions (lists/dicts).
    We store them in global_functions/global_variables with keys like:
      "some_package.sub_module.my_func" -> ast.FunctionDef
    """
    with open(filepath, "r", encoding="utf-8") as f:
        source = f.read()
    tree = ast.parse(source, filename=filepath)

    # We'll define a small visitor:
    class TopLevelCollector(ast.NodeVisitor):
        def visit_FunctionDef(self, node: ast.FunctionDef):
            # Build key
            fq_key = f"{module_name}.{node.name}"
            global_functions[fq_key] = node
            self.generic_visit(node)

        def visit_AsyncFunctionDef(self, node: ast.AsyncFunctionDef):
            fq_key = f"{module_name}.{node.name}"
            global_functions[fq_key] = node
            self.generic_visit(node)

        def visit_Assign(self, node: ast.Assign):
            # If it's a top-level assignment: e.g. my_var = [1,2,3]
            # We'll store it if it's a list or dict
            if isinstance(node.parent, ast.Module):
                # We only want top-level assigns, not inside a class or function
                if len(node.targets) == 1 and isinstance(node.targets[0], ast.Name):
                    var_name = node.targets[0].id
                    val = node.value
                    if isinstance(val, (ast.List, ast.Dict)):
                        fq_key = f"{module_name}.{var_name}"
                        global_variables[fq_key] = val
            self.generic_visit(node)

        # We also override generic_visit to set a parent pointer so we can detect top-level
        def generic_visit(self, node):
            for child in ast.iter_child_nodes(node):
                child.parent = node  # store a pointer to parent
                self.visit(child)

    collector = TopLevelCollector()
    collector.visit(tree)


In [3]:
build_global_registry("./Gladiator2", "./Gladiator2")

In [4]:
class ClassInstanceTracker(ast.NodeVisitor):
    """
    A node visitor that:
      1. Collects import information to map local names to fully-qualified paths.
      2. Finds instantiations of a target class (by fully-qualified name).
      3. Tracks method calls on those instances, capturing arguments.
      4. Adds extra logic for a 'special method':
         - If called with 2 positional args, the second is the name of a function -> gather return expressions
         - If called with 3 positional args, the third is a literal list/dict or a variable referencing one -> gather values
    """

    def __init__(self, graph_class_fqcn: str, add_conditional_edges_method_name: str, add_node_method_name: str):
        """
        :param graph_class_fqcn: The fully qualified class name of the Graph class to look for.
        :param add_conditional_edges_method_name: The method name that, when called, triggers addition of conditional edges.
        :param add_node_method_name: The method name that, when called, triggers addition of a node.
        """
        self.graph_class_fqcn = graph_class_fqcn
        self.add_conditional_edges_method_name = add_conditional_edges_method_name
        self.add_node_method_name = add_node_method_name

        # Maintain a mapping of local import aliases to fully qualified paths.
        # E.g., "import examples as ex" => self.import_aliases["ex"] = "examples"
        self.import_aliases: Dict[str, str] = {}

        # For "from examples.graph import Graph", store fully qualified references:
        # e.g., self.import_aliases_fully["Graph"] = "examples.graph.Graph"
        self.import_aliases_fully: Dict[str, str] = {}

        # We'll accumulate method calls in this structure:
        # {
        #     instance_name: {
        #         method_name: [
        #             {
        #                 "positional": [...],
        #                 "keyword": {...},
        #                 # possibly "function_returns", "list_values", or "dict_values"
        #             },
        #             ...
        #         ]
        #     },
        #     ...
        # }
        self.instance_methods: Dict[str, Dict[str, List[Dict[str, Any]]]] = {}

        # Track which variables are instances of our target class.
        # E.g., if we see "g = Graph()", then self.variable_is_target_instance["g"] = True
        self.variable_is_target_instance: Dict[str, bool] = {}

        # Keep track of all top-level functions (both async and regular):
        # function name -> (ast.FunctionDef or ast.AsyncFunctionDef)
        self.function_defs: Dict[str, Union[ast.FunctionDef, ast.AsyncFunctionDef]] = {}

        # Track variables that hold a literal list or dict, so that if a method is called
        # with that variable, we can retrieve its contents. E.g.:
        # my_list = [1,2,3] => self.variable_values["my_list"] = ast.List(...)
        self.variable_values: Dict[str, Union[ast.List, ast.Dict]] = {}

    # -------------------------------------------------------------------------
    #                           AST Visitor Methods
    # -------------------------------------------------------------------------

    def visit_Import(self, node: ast.Import) -> Any:
        """
        Handle statements like:
          import examples
          import examples.graph as ex
        We store the local alias => real module path in self.import_aliases.
        """
        for alias in node.names:
            local_name = alias.asname if alias.asname else alias.name
            # e.g. "examples.graph" or "examples"
            self.import_aliases[local_name] = alias.name

        self.generic_visit(node)

    def visit_ImportFrom(self, node: ast.ImportFrom) -> Any:
        """
        Handle statements like:
          from examples.graph import Graph
          from examples import graph as gph
        We store the local name => fully qualified path in self.import_aliases_fully.
        """
        # node.module might be None for relative imports like "from . import Something"
        if node.module is None:
            # Skip or handle relative imports if needed
            self.generic_visit(node)
            return

        base_module = node.module  # e.g. "examples.graph"
        for alias in node.names:
            local_name = alias.asname if alias.asname else alias.name
            # e.g., local_name "Graph" -> "examples.graph.Graph"
            self.import_aliases_fully[local_name] = f"{base_module}.{alias.name}"

        self.generic_visit(node)

    def visit_FunctionDef(self, node: ast.FunctionDef) -> Any:
        """
        Record the top-level function definitions for later retrieval.
        """
        self.function_defs[node.name] = node
        # Continue visiting its body, because it may have calls we care about
        self.generic_visit(node)

    def visit_AsyncFunctionDef(self, node: ast.AsyncFunctionDef) -> Any:
        """
        Same as visit_FunctionDef, but for async functions.
        """
        self.function_defs[node.name] = node
        self.generic_visit(node)

    def visit_Assign(self, node: ast.Assign) -> Any:
        """
        Handle assignments, e.g.:
            g = Graph(...)
            my_list = [1, 2, 3]
            my_dict = {"foo": 42}
        We identify if the assigned value is a call to our target class
        or a literal list/dict for which we store references in variable_values.
        """
        # If there's exactly one target, and it's a Name, e.g., "x = ..."
        if len(node.targets) == 1 and isinstance(node.targets[0], ast.Name):
            var_name = node.targets[0].id

            # If the right side is a Call node, check if it instantiates our target class.
            if isinstance(node.value, ast.Call):
                if self._call_is_target_class(node.value):
                    self.variable_is_target_instance[var_name] = True

            # If the right side is a literal list or dict, store it for later.
            if isinstance(node.value, (ast.List, ast.Dict)):
                self.variable_values[var_name] = node.value
            else:
                # If we reassign the variable to something else, remove from variable_values
                if var_name in self.variable_values:
                    del self.variable_values[var_name]

        self.generic_visit(node)

    # THIS CAN PROBABLY BE REMOVED
    def visit_Expr(self, node: ast.Expr) -> Any:
        """
        If there's a bare expression that directly calls something,
        we check if it's instantiating our target class (rare, but possible).
        """
        if isinstance(node.value, ast.Call):
            self._call_is_target_class(node.value)
        self.generic_visit(node)

    def visit_Call(self, node: ast.Call) -> Any:
        """
        For method calls on an instance: e.g., g.add_node(...).
        If 'g' is known to be an instance of our target class, we record the call.

        Also, if the method name matches our 'special method', we run extra analysis:
        - 2 args => second is a function name => gather return expressions
        - 3 args => third is a literal list/dict or a variable referencing one => gather values
        """
        # The 'func' of a Call might be an Attribute like g.add_node
        if isinstance(node.func, ast.Attribute):
            # node.func.value might be a Name, e.g., 'g'
            if isinstance(node.func.value, ast.Name):
                instance_name = node.func.value.id

                # Check if that variable is an instance of our target class
                if self.variable_is_target_instance.get(instance_name, False):
                    # This is a method call on our target instance
                    method_name = node.func.attr

                    call_record = {
                        "positional": [self._stringify_ast_node(a) for a in node.args],
                        "keyword": {
                            kw.arg: self._stringify_ast_node(kw.value)
                            for kw in node.keywords
                            if kw.arg is not None
                        }
                    }

                    # If it's the special method, do the extended analysis
                    if method_name == self.add_conditional_edges_method_name:
                        self._process_add_conditional_edges_method_call(node, call_record)

                    if method_name == self.add_node_method_name:
                        self._handle_add_node_argument(node, call_record)

                    # Store the call record in instance_methods
                    if instance_name not in self.instance_methods:
                        self.instance_methods[instance_name] = {}
                    if method_name not in self.instance_methods[instance_name]:
                        self.instance_methods[instance_name][method_name] = []

                    self.instance_methods[instance_name][method_name].append(call_record)

        # Continue traversing the AST
        self.generic_visit(node)

    # -------------------------------------------------------------------------
    #                           Helper Methods
    # -------------------------------------------------------------------------

    def _handle_add_node_argument(self, call_node: ast.Call, call_record: dict) -> None:
        """
        If my_special_method has 1 positional arg => analyze that arg.
        If my_special_method has 2 positional args => analyze the second arg.
        We store the resulting info in call_record["node_definition_argument_info"].
        """
        num_pos = len(call_node.args)
        if num_pos == 1:
            # The single argument is call_node.args[0]
            info = self._analyze_argument(call_node.args[0])
            call_record["node_definition_argument_info"] = info

        elif num_pos == 2:
            # The second argument is call_node.args[1]
            info = self._analyze_argument(call_node.args[1])
            call_record["node_definition_argument_info"] = info


    def _analyze_argument(self, node: ast.AST) -> Dict[str, Optional[str]]:
        """
        Return a dictionary describing the argument:
        - "original": how it appears in the code
        - "fq_name": best guess fully qualified name (if we can resolve)

        The argument might be:
        - A Name referencing a local or imported function/variable
        - An Attribute referencing a local or imported function/variable
        - A Call referencing a local or imported function/class
        - (You could extend for other node types if needed.)
        """
        result: Dict[str, Optional[str]] = {
            "original": self._stringify_ast_node(node),
            "fq_name": None
        }

        # -----------------------------------------
        # 1) If it's a bare name (like "some_func")
        # -----------------------------------------
        if isinstance(node, ast.Name):
            fq_name = self._resolve_fq_name(node)
            result["fq_name"] = fq_name

        # -------------------------------------------------
        # 2) If it's an attribute, e.g. "some_import.func"
        # -------------------------------------------------
        elif isinstance(node, ast.Attribute):
            # e.g. node might represent "some_import.function"
            fq_name = self._resolve_fq_name(node)
            result["fq_name"] = fq_name

        # -----------------------------------------------------------------------------------
        # 3) If it's a call, e.g. "some_import.function(...)" or "local_func(...)" or "Name()"
        # -----------------------------------------------------------------------------------
        elif isinstance(node, ast.Call):
            # The function called might be a Name or an Attribute
            func = node.func
            if isinstance(func, (ast.Name, ast.Attribute)):
                fq_name = self._resolve_fq_name(func)
                result["fq_name"] = fq_name

            else:
                # e.g. calling a lambda or subscript, etc.
                result["fq_name"] = result["original"]

        # If it's something else (List, Dict, Subscript, etc.), we skip. Extend as needed.

        return result


    def _record_function_info(self, arg_node: ast.AST, call_record: dict) -> None:
        """
        Takes the node believed to represent a function reference, tries to figure out
        if it's local or imported. We'll store info in call_record:
            call_record["function_name"]
            call_record["function_source"]
            call_record["function_fq"]  # fully qualified if we can determine
        """

        # We'll do a small helper function to resolve the 'function reference' to an FQ name.
        # Then see if it's local or imported or unknown.
        function_fq_name = self._resolve_function_reference(arg_node)

        if function_fq_name is None:
            # We couldn't figure out a function name from this node
            return

        call_record["function_name"] = function_fq_name  # or short name if you prefer

        # Now decide if it's local or imported
        # If the short name portion is in self.function_defs => local
        # If it matches an import => imported
        short_name = function_fq_name.split(".")[-1]

        if short_name in self.function_defs:
            call_record["function_source"] = "local"
        else:
            call_record["function_source"] = "imported"

        call_record["function_fq"] = function_fq_name

    def _resolve_function_reference(self, node: ast.AST) -> Union[str, None]:
        """
        Try to interpret 'node' as a reference to a function.
        Possible cases:
          - A bare Name, e.g. 'some_func'
          - An attribute chain, e.g. 'utils.func'
          - Something else (Call, Subscript, etc.)

        We'll reuse our _resolve_fq_name logic if it's a Name or an Attribute.
        If we can't handle it, return None.
        """
        if isinstance(node, ast.Name):
            # e.g. 'some_func'
            # Check if it appears in import_aliases_fully or import_aliases
            # or just return node.id as is
            fq = self._resolve_fq_name(node)
            return fq

        elif isinstance(node, ast.Attribute):
            # e.g. 'utils.func'
            fq = self._resolve_fq_name(node)
            return fq

        # If it's something else (like a lambda, a call, a subscript),
        # we won't treat it as a function reference for now
        return None
    


    def _process_add_conditional_edges_method_call(self, call_node: ast.Call, call_record: dict) -> None:
        """
        If the method is our special method, do extra checks. Now we handle both
        positional and keyword arguments in the source order to determine
        the 'second' or 'third' argument.

        1) If there are exactly 2 total arguments, treat the second as a function name => gather returns.
        2) If there are exactly 3 total arguments, treat the third as a list/dict literal or variable => gather values.
        """
        # Combine all arguments (positional + keyword) in source order.
        # The 'value' of each keyword is the actual expression.
        all_args_in_order = self._get_all_args_in_call(call_node)

        if len(all_args_in_order) == 2:
            # The second argument is presumably a function name
            second_arg = all_args_in_order[1]
            if isinstance(second_arg, ast.Name):
                func_name = second_arg.id
                # If we have that function in function_defs, gather return expressions
                if func_name in self.function_defs:
                    returns = self._get_return_expressions(self.function_defs[func_name])
                    call_record["path"] = {
                        "function_returns": returns
                    }
                elif self._resolve_fq_name(second_arg) in global_functions:
                    returns = self._get_return_expressions(global_functions[self._resolve_fq_name(second_arg)])
                    call_record["path"] = {
                        "function_returns": returns
                    }
                else:
                    # fq_name = self._resolve_fq_name(second_arg)
                    # call_record["path"] = {
                    #     "function_fq_name": fq_name
                    # }
                    pass

        elif len(all_args_in_order) == 3:
            # The third argument might be a list/dict literal or a variable referencing one
            third_arg = all_args_in_order[2]

            # Directly a list literal
            if isinstance(third_arg, ast.List):
                list_vals = [self._stringify_ast_node(elt) for elt in third_arg.elts]
                call_record["path_map"] = {
                    "list_values": list_vals
                }

            # Directly a dict literal
            elif isinstance(third_arg, ast.Dict):
                dict_vals = [self._stringify_ast_node(v) for v in third_arg.values]
                call_record["path_map"] = {
                    "dict_values": dict_vals
                }

            # A variable referencing a list/dict literal
            elif isinstance(third_arg, ast.Name):
                var_name = third_arg.id
                if var_name in self.variable_values:
                    val_node = self.variable_values[var_name]
                    if isinstance(val_node, ast.List):
                        list_vals = [self._stringify_ast_node(elt) for elt in val_node.elts]
                        call_record["path_map"] = {
                            "list_values": list_vals
                        }
                    elif isinstance(val_node, ast.Dict):
                        dict_vals = [self._stringify_ast_node(v) for v in val_node.values]
                        call_record["path_map"] = {
                            "dict_values": dict_vals
                        }
                elif self._resolve_fq_name(third_arg) in global_variables:
                    val_node = global_variables[self._resolve_fq_name(third_arg)]
                    if isinstance(val_node, ast.List):
                        list_vals = [self._stringify_ast_node(elt) for elt in val_node.elts]
                        call_record["path_map"] = {
                            "list_values": list_vals
                        }
                    elif isinstance(val_node, ast.Dict):
                        dict_vals = [self._stringify_ast_node(v) for v in val_node.values]
                        call_record["path_map"] = {
                            "dict_values": dict_vals
                        }

                else:
                    # fq_name = self._resolve_fq_name(third_arg)
                    # call_record["path_map"] = {
                    #     "map_fq_name": fq_name
                    # }
                    pass


    def _call_is_target_class(self, call_node: ast.Call) -> bool:
        """
        Checks if the call instantiates our target class, e.g. Graph(...) when we want
        "examples.graph.Graph" or ex.graph.Graph(...) if ex => "examples".
        We do this by resolving the fully qualified name of the callee and comparing
        against self.graph_class_fqcn.
        """
        fq_name = self._resolve_fq_name(call_node.func)
        return (fq_name == self.graph_class_fqcn)

    def _resolve_fq_name(self, node: Union[ast.Name, ast.Attribute]) -> str:
        """
        Attempt to resolve a node (Name or Attribute chain) to a fully qualified name.
        E.g., if node is "ex.graph.Graph" and ex => "examples", we get "examples.graph.Graph".
        If it's just "Graph", and we have "Graph" => "examples.graph.Graph" in import_aliases_fully,
        we return that. Otherwise, we return the best guess or the bare name.
        """
        if isinstance(node, ast.Name):
            # Possibly in self.import_aliases_fully or self.import_aliases
            if node.id in self.import_aliases_fully:
                return self.import_aliases_fully[node.id]
            else:
                # If not, check if there's a top-level import alias
                return self.import_aliases.get(node.id, node.id)
        elif isinstance(node, ast.Attribute):
            # Build up from base, then dot + attr
            base_fq = self._resolve_fq_name(node.value)
            if base_fq:
                return base_fq + "." + node.attr
            else:
                # Fallback: just the attribute name
                return node.attr
        return ""

    def _stringify_ast_node(self, node: ast.AST) -> str:
        """
        Convert an AST node into a string for storing in "positional"/"keyword" argument lists
        or for analyzing returns. We do a few special cases for clarity:
          - ast.Constant => repr(value)
          - ast.List => "ListLiteral([...])"
          - ast.Dict => "DictLiteral({...})"
          - ast.Name => variable name
          - ast.Call => "Call(...ast dump...)"
          - Otherwise => ast.dump(node)
        """
        if isinstance(node, ast.Constant):
            if isinstance(node.value, str):
                return node.value
            return repr(node.value)
        elif isinstance(node, ast.Name):
            return node.id
        elif isinstance(node, ast.Call):
            return f"Call({ast.dump(node)})"
        elif isinstance(node, ast.List):
            # Convert elements recursively
            return "ListLiteral([" + ", ".join(self._stringify_ast_node(elt) for elt in node.elts) + "])"
        elif isinstance(node, ast.Dict):
            # Convert key-value pairs recursively
            keys = [self._stringify_ast_node(k) for k in node.keys]
            vals = [self._stringify_ast_node(v) for v in node.values]
            pairs_str = ", ".join(f"{k}: {v}" for k, v in zip(keys, vals))
            return f"DictLiteral({{{pairs_str}}})"
        else:
            # Fallback: the raw AST dump
            return ast.dump(node)
        
    def _get_all_args_in_call(self, call_node: ast.Call) -> List[ast.expr]:
        """
        Returns a list of all arguments (positional first, then each
        keyword argument's value) in the **exact order** they appear in the source.
        
        Example:
          special_method(10, foo='bar', my_list=[1,2,3])
          => [ast.Constant(value=10), ast.Constant(value='bar'), ast.List(elts=[...])]
        
        This helps unify the logic for "the second argument," "the third argument," etc.
        """
        combined = []
        combined.extend(call_node.args)

        for kw in call_node.keywords:
            combined.append(kw.value)
        return combined

    def _get_return_expressions(
        self,
        func_def: Union[ast.FunctionDef, ast.AsyncFunctionDef]
    ) -> List[str]:
        """
        Traverse the body of a function definition and collect stringified return expressions.
        For every 'return X', we gather the stringified form of X.
        """

        returns = []

        class ReturnCollector(ast.NodeVisitor):
            """
            Inner visitor class that visits Return nodes in the function body.
            We store them in the outer 'returns' list.
            """

            def __init__(inner_self, outer_self):
                super().__init__()
                inner_self.outer = outer_self
                inner_self.variables: Dict[str, str] = {}

            def visit_Return(inner_self, return_node: ast.Return):
                if return_node.value is not None and isinstance(return_node.value, ast.Name):
                    if return_node.value.id in inner_self.variables:
                        returns.extend(inner_self.variables[return_node.value.id])
                    else:
                        returns.append(inner_self.outer._stringify_ast_node(return_node.value))
                if return_node.value is not None and isinstance(return_node.value, ast.Constant):
                    # Call the outer class's _stringify_ast_node to convert the expression
                    returns.append(return_node.value.value)

            def visit_Assign(inner_self, node: ast.Assign) -> Any:
                """
                Handle assignments in case code is like:
                if ...:
                    next_node = "node1"
                else:
                    next_node = "node2"

                return next_node
                """
                # If there's exactly one target, and it's a Name, e.g., "x = ..."
                if len(node.targets) == 1 and isinstance(node.targets[0], ast.Name):
                    var_name = node.targets[0].id

                    # If the right side is a literal list or dict, store it for later.
                    if isinstance(node.value, ast.Constant):
                        # and node.value accounts for None
                        if var_name not in inner_self.variables and node.value.value is not None:
                            inner_self.variables[var_name] = [node.value.value]
                        elif node.value.value:
                            inner_self.variables[var_name].append(node.value.value)
                    elif isinstance(node.value, ast.Name):
                        if var_name not in inner_self.variables and node.value.value is not None:
                            inner_self.variables[var_name] = [inner_self.outer._stringify_ast_node(node.value)]
                        elif node.value:
                            inner_self.variables[var_name].append(inner_self.outer._stringify_ast_node(node.value))


        # Instantiate the ReturnCollector, giving it a reference to our outer class (self)
        ReturnCollector(self).visit(func_def)
        return returns


def parse_python_file(
    filepath: str,
    target_class: str,
    add_conditional_edges_method_name: str,
    add_node_method_name: str
) -> Dict[str, Dict[str, List[Dict[str, Any]]]]:
    """
    Parse a single Python file for:
      - Instantiations of 'target_class'
      - Method calls on those instances
      - Special logic for the specified 'add_conditional_edges_method_name'

    :param filepath: path to a .py file
    :param target_class: fully-qualified class name we want to detect
    :param add_conditional_edges_method_name: name of the method that triggers extra analysis
    :return: structure summarizing the method calls on each instance
    """
    with open(filepath, "r", encoding="utf-8") as f:
        source = f.read()

    # Parse the entire file into an AST
    tree = ast.parse(source, filename=filepath)

    # Create our tracking visitor
    tracker = ClassInstanceTracker(target_class, add_conditional_edges_method_name, add_node_method_name)

    # Visit the AST
    tracker.visit(tree)

    # Return the final dictionary of instance methods
    return tracker.instance_methods


def walk_directory_and_parse(
    root_dir: str,
    target_class: str,
    add_conditional_edges_method_name: str,
    add_node_method_name: str
) -> Dict[str, Dict[str, List[Dict[str, Any]]]]:
    """
    Recursively walk a directory (root_dir), parse each .py file,
    and merge results for the given target class & special method logic.

    :param root_dir: The directory path to recursively walk
    :param target_class: Fully qualified class name, e.g. "examples.graph.Graph"
    :param add_conditional_edges_method_name: The method name that triggers extra analysis
    :param add_node_method_name: The method name that triggers extra analysis
    :return: Combined dictionary for all .py files under root_dir
    """
    combined_results: Dict[str, Dict[str, List[Dict[str, Any]]]] = {}

    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(".py"):
                fullpath = os.path.join(dirpath, filename)
                file_results = parse_python_file(fullpath, target_class, add_conditional_edges_method_name, add_node_method_name)

                # Merge this file's results into the overall dictionary
                for instance_name, methods_dict in file_results.items():
                    if instance_name not in combined_results:
                        combined_results[instance_name] = {}
                    for method_name, calls in methods_dict.items():
                        if method_name not in combined_results[instance_name]:
                            combined_results[instance_name][method_name] = []
                        combined_results[instance_name][method_name].extend(calls)
                        combined_results[instance_name]["filepath"] = fullpath

    return combined_results

# root_directory = "./customer_service"
# graph_class_fqcn = "langgraph.graph.StateGraph"
# add_conditional_edges_method_name = "add_conditional_edges"
# add_node_method_name = "add_node"

root_directory = "./Gladiator2"
graph_class_fqcn = "langgraph.graph.StateGraph"
add_conditional_edges_method_name = "add_conditional_edges"
add_node_method_name = "add_node"

results = walk_directory_and_parse(root_directory, graph_class_fqcn, add_conditional_edges_method_name, add_node_method_name)

import pprint
pprint.pprint(results)


{}


In [5]:
def extract_custom_tools_with_ast(file_content, file_path):
    custom_tools = []
    tree = ast.parse(file_content)

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            decorator_names = []
            for decorator in node.decorator_list:
                if isinstance(decorator, ast.Name):
                    decorator_names.append(decorator.id)
                elif isinstance(decorator, ast.Call):
                    if isinstance(decorator.func, ast.Name):
                        decorator_names.append(decorator.func.id)
                    elif isinstance(decorator.func, ast.Attribute):
                        decorator_names.append(decorator.func.attr)
            
            if "tool" in decorator_names:
                custom_tools.append({
                    "name": node.name,
                    "filepath": file_path
                })
    
    return custom_tools
    


results = {}
def parse_all_custom_tools_from_directory(directory_path):
    all_custom_tools = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    file_content = f.read()
                    custom_tools = extract_custom_tools_with_ast(file_content, file_path)
                    for single_custom_tool in custom_tools:
                        all_custom_tools.append(single_custom_tool)
    
    return all_custom_tools

In [6]:
def extract_imports_with_ast(file_content):
    imports = []
    tree = ast.parse(file_content)

    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                full_import = alias.name
                imports.append(full_import)
        
        elif isinstance(node, ast.ImportFrom):
            module = node.module
            for alias in node.names:
                full_import = f"{module}.{alias.name}"
                imports.append(full_import)
    
    return imports

results = {}
def parse_all_imports_from_directory(directory_path):
    all_imports = set()
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    file_content = f.read()
                    imports = extract_imports_with_ast(file_content)
                    for single_import in imports:
                        all_imports.add(single_import)
    
    return all_imports

In [7]:
predefined_tools_dict = {'AINetwork Toolkit': {'imports': 'from langchain_community.agent_toolkits.ainetwork.toolkit import AINetworkToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.ainetwork.toolkit.AINetworkToolkit']}, 'Alpha Vantage': {'imports': 'from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.alpha_vantage.AlphaVantageAPIWrapper']}, 'Amadeus Toolkit': {'imports': 'from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.amadeus.toolkit.AmadeusToolkit']}, 'ArXiv': {'imports': 'from langchain_community.utilities import ArxivAPIWrapper\nor\nfrom langchain.agents import load_tools\ntools = load_tools(\n    ["arxiv"],\n)', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.ArxivAPIWrapper', 'langchain.agents.load_tools']}, 'AskNews': {'imports': 'from langchain_community.retrievers import AskNewsRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.AskNewsRetriever']}, 'AWS Lambda': {'imports': 'from langchain.agents import load_tools\ntools = load_tools(\n    ["awslambda"],\n)', 'category': 'GENERAL', 'import_list': ['langchain.agents.load_tools']}, 'Azure AI Services Toolkit': {'imports': 'from langchain_community.agent_toolkits import AzureAiServicesToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.AzureAiServicesToolkit']}, 'Azure Cognitive Services Toolkit': {'imports': 'from langchain_community.agent_toolkits import AzureCognitiveServicesToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.AzureCognitiveServicesToolkit']}, 'Azure Container Apps dynamic sessions': {'imports': 'from langchain_azure_dynamic_sessions import SessionsPythonREPLTool', 'category': 'GENERAL', 'import_list': ['langchain_azure_dynamic_sessions.SessionsPythonREPLTool']}, 'Shell (bash)': {'imports': 'from langchain_community.tools import ShellTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.ShellTool']}, 'Bearly Code Interpreter': {'imports': 'from langchain_community.tools import BearlyInterpreterTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.BearlyInterpreterTool']}, 'Bing Search': {'imports': 'from langchain_community.utilities import BingSearchAPIWrapper\nor\nfrom langchain_community.tools.bing_search import BingSearchResults\n', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.BingSearchAPIWrapper', 'langchain_community.tools.bing_search.BingSearchResults']}, 'Brave Search': {'imports': 'from langchain_community.document_loaders import BraveSearchLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BraveSearchLoader']}, 'Cassandra Database Toolkit': {'imports': 'from langchain_community.agent_toolkits.cassandra_database.toolkit import CassandraDatabaseToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.cassandra_database.toolkit.CassandraDatabaseToolkit']}, 'CDP': {'imports': 'from cdp_langchain.agent_toolkits import CdpToolkit\nor\nfrom cdp_langchain.utils import CdpAgentkitWrapper\n', 'category': 'GENERAL', 'import_list': ['cdp_langchain.agent_toolkits.CdpToolkit', 'cdp_langchain.utils.CdpAgentkitWrapper']}, 'ClickUp Toolkit': {'imports': 'from langchain_community.agent_toolkits.clickup.toolkit import ClickupToolkit\nor\nfrom langchain_community.utilities.clickup import ClickupAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.clickup.toolkit.ClickupToolkit', 'langchain_community.utilities.clickup.ClickupAPIWrapper']}, 'Cogniswitch Toolkit': {'imports': 'from langchain_community.agent_toolkits import CogniswitchToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.CogniswitchToolkit']}, 'Connery Toolkit and Tools': {'imports': 'from langchain_community.agent_toolkits.connery import ConneryToolkit\nor\nfrom langchain_community.tools.connery import ConneryService', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.connery.ConneryToolkit', 'langchain_community.tools.connery.ConneryService']}, 'Dall-E Image Generator': {'imports': 'from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["dalle-image-generator"])\n', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.dalle_image_generator.DallEAPIWrapper', 'langchain.agents.load_tools']}, 'Dappier': {'imports': 'from langchain_dappier import DappierRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_dappier.DappierRetriever']}, 'Databricks Unity Catalog (UC)': {'imports': 'from langchain_community.tools.databricks import UCFunctionToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.databricks.UCFunctionToolkit']}, 'DataForSEO': {'imports': 'from langchain_community.utilities.dataforseo_api_search import DataForSeoAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.dataforseo_api_search.DataForSeoAPIWrapper']}, 'Dataherald': {'imports': 'from langchain_community.utilities.dataherald import DataheraldAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.dataherald.DataheraldAPIWrapper']}, 'DuckDuckGo Search': {'imports': 'from langchain_community.tools import DuckDuckGoSearchRun\nor\nfrom langchain_community.tools import DuckDuckGoSearchResults', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.DuckDuckGoSearchRun', 'langchain_community.tools.DuckDuckGoSearchResults']}, 'E2B Data Analysis': {'imports': 'from langchain_community.tools import E2BDataAnalysisTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.E2BDataAnalysisTool']}, 'Eden AI': {'imports': 'from langchain_community.chat_models.edenai import ChatEdenAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.edenai.ChatEdenAI']}, 'Eleven Labs Text2Speech': {'imports': 'from langchain_community.tools import ElevenLabsText2SpeechTool\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["eleven_labs_text2speech"])', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.ElevenLabsText2SpeechTool', 'langchain.agents.load_tools']}, 'Exa Search': {'imports': 'from exa_py import Exa', 'category': 'GENERAL', 'import_list': ['exa_py.Exa']}, 'File System': {'imports': 'from langchain_community.agent_toolkits import FileManagementToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.FileManagementToolkit']}, 'FinancialDatasets Toolkit': {'imports': 'from langchain_community.agent_toolkits.financial_datasets.toolkit import FinancialDatasetsToolkit\nor\nfrom langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.financial_datasets.toolkit.FinancialDatasetsToolkit', 'langchain_community.utilities.financial_datasets.FinancialDatasetsAPIWrapper']}, 'FMP Data': {'imports': 'from langchain_fmp_data import FMPDataTool\nor\nfrom langchain_fmp_data import FMPDataToolkit', 'category': 'GENERAL', 'import_list': ['langchain_fmp_data.FMPDataTool', 'langchain_fmp_data.FMPDataToolkit']}, 'Github Toolkit': {'imports': 'from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit\nor\nfrom langchain_community.utilities.github import GitHubAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.github.toolkit.GitHubToolkit', 'langchain_community.utilities.github.GitHubAPIWrapper']}, 'Gitlab Toolkit': {'imports': 'from langchain_community.agent_toolkits.gitlab.toolkit import GitLabToolkit\nor\nfrom langchain_community.utilities.gitlab import GitLabAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.gitlab.toolkit.GitLabToolkit', 'langchain_community.utilities.gitlab.GitLabAPIWrapper']}, 'Gmail Toolkit': {'imports': 'from langchain_google_community import GmailToolkit', 'category': 'GENERAL', 'import_list': ['langchain_google_community.GmailToolkit']}, 'Golden Query': {'imports': 'from langchain_community.utilities.golden_query import GoldenQueryAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.golden_query.GoldenQueryAPIWrapper']}, 'Google Books': {'imports': 'from langchain_community.tools.google_books import GoogleBooksQueryRun\nor\nfrom langchain_community.utilities.google_books import GoogleBooksAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_books.GoogleBooksQueryRun', 'langchain_community.utilities.google_books.GoogleBooksAPIWrapper']}, 'Google Cloud Text-to-Speech': {'imports': 'from langchain_google_community import TextToSpeechTool', 'category': 'GENERAL', 'import_list': ['langchain_google_community.TextToSpeechTool']}, 'Google Drive': {'imports': 'from langchain_google_community import GoogleDriveLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_community.GoogleDriveLoader']}, 'Google Finance': {'imports': 'from langchain_community.tools.google_finance import GoogleFinanceQueryRun\nor\nfrom langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["google-finance"])', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_finance.GoogleFinanceQueryRun', 'langchain_community.utilities.google_finance.GoogleFinanceAPIWrapper', 'langchain.agents.load_tools']}, 'Google Imagen': {'imports': 'from langchain_google_vertexai.vision_models import VertexAIImageGeneratorChat\nor\nfrom langchain_google_vertexai.vision_models import VertexAIImageEditorChat\nor\nfrom langchain_google_vertexai import VertexAIImageCaptioning\nor\nfrom langchain_google_vertexai import VertexAIVisualQnAChat', 'category': 'GENERAL', 'import_list': ['langchain_google_vertexai.vision_models.VertexAIImageGeneratorChat', 'langchain_google_vertexai.vision_models.VertexAIImageEditorChat', 'langchain_google_vertexai.VertexAIImageCaptioning', 'langchain_google_vertexai.VertexAIVisualQnAChat']}, 'Google Jobs': {'imports': 'from langchain_community.tools.google_jobs import GoogleJobsQueryRun\nor\nfrom langchain_community.utilities.google_jobs import GoogleJobsAPIWrapper\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["google-jobs"], llm=llm)', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_jobs.GoogleJobsQueryRun', 'langchain_community.utilities.google_jobs.GoogleJobsAPIWrapper', 'langchain.agents.load_tools']}, 'Google Lens': {'imports': 'from langchain_community.tools.google_lens import GoogleLensQueryRun\nor\nfrom langchain_community.utilities.google_lens import GoogleLensAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_lens.GoogleLensQueryRun', 'langchain_community.utilities.google_lens.GoogleLensAPIWrapper']}, 'Google Places': {'imports': 'from langchain_community.tools import GooglePlacesTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.GooglePlacesTool']}, 'Google Scholar': {'imports': 'from langchain_community.tools.google_scholar import GoogleScholarQueryRun\nor\nfrom langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_scholar.GoogleScholarQueryRun', 'langchain_community.utilities.google_scholar.GoogleScholarAPIWrapper']}, 'Google Search': {'imports': 'from langchain_google_community import GoogleSearchAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_google_community.GoogleSearchAPIWrapper']}, 'Google Serper': {'imports': 'from langchain_community.utilities import GoogleSerperAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.GoogleSerperAPIWrapper']}, 'Google Trends': {'imports': 'from langchain_community.tools.google_trends import GoogleTrendsQueryRun\nor\nfrom langchain_community.utilities.google_trends import GoogleTrendsAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.google_trends.GoogleTrendsQueryRun', 'langchain_community.utilities.google_trends.GoogleTrendsAPIWrapper']}, 'Gradio': {'imports': 'from gradio_tools.tools import ImageCaptioningTool\nor\nfrom gradio_tools.tools import StableDiffusionPromptGeneratorTool\nor\nfrom gradio_tools.tools import StableDiffusionTool\nor\nfrom gradio_tools.tools import TextToVideoTool', 'category': 'GENERAL', 'import_list': ['gradio_tools.tools.ImageCaptioningTool', 'gradio_tools.tools.StableDiffusionPromptGeneratorTool', 'gradio_tools.tools.StableDiffusionTool', 'gradio_tools.tools.TextToVideoTool']}, 'GraphQL': {'imports': 'from langchain.agents import load_tools\ntools = load_tools(\n    ["graphql"],\n)', 'category': 'GENERAL', 'import_list': ['langchain.agents.load_tools']}, 'HuggingFace Hub Tools': {'imports': 'from langchain_community.agent_toolkits.load_tools import load_huggingface_tool', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.load_tools.load_huggingface_tool']}, 'Human as a tool': {'imports': 'from langchain.agents import load_tools\ntools = load_tools(\n    ["human"],\n)', 'category': 'GENERAL', 'import_list': ['langchain.agents.load_tools']}, 'IFTTT WebHooks': {'imports': 'from langchain_community.tools.ifttt import IFTTTWebhook', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.ifttt.IFTTTWebhook']}, 'Infobip': {'imports': 'from langchain_community.utilities.infobip import InfobipAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.infobip.InfobipAPIWrapper']}, 'Ionic Shopping Tool': {'imports': 'from ionic_langchain.tool import IonicTool', 'category': 'GENERAL', 'import_list': ['ionic_langchain.tool.IonicTool']}, 'Jina Search': {'imports': 'from langchain_community.tools import JinaSearch', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.JinaSearch']}, 'Jira Toolkit': {'imports': 'from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit\nor\nfrom langchain_community.utilities.jira import JiraAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.jira.toolkit.JiraToolkit', 'langchain_community.utilities.jira.JiraAPIWrapper']}, 'JSON Toolkit': {'imports': 'from langchain_community.agent_toolkits import JsonToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.JsonToolkit']}, 'Lemon Agent': {'imports': 'from lemonai import execute_workflow', 'category': 'GENERAL', 'import_list': ['lemonai.execute_workflow']}, 'LinkupSearchTool': {'imports': 'from langchain_linkup import LinkupSearchTool', 'category': 'GENERAL', 'import_list': ['langchain_linkup.LinkupSearchTool']}, 'Memorize': {'imports': 'from langchain.agents import load_tools\ntools = load_tools(["memorize"], llm=llm)', 'category': 'GENERAL', 'import_list': ['langchain.agents.load_tools']}, 'Mojeek Search': {'imports': 'from langchain_community.tools import MojeekSearch', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.MojeekSearch']}, 'MultiOn Toolkit': {'imports': 'from langchain_community.agent_toolkits import MultionToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.MultionToolkit']}, 'NASA Toolkit': {'imports': 'from langchain_community.agent_toolkits.nasa.toolkit import NasaToolkit\nor\nfrom langchain_community.utilities.nasa import NasaAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.nasa.toolkit.NasaToolkit', 'langchain_community.utilities.nasa.NasaAPIWrapper']}, 'Nuclia': {'imports': 'from langchain_community.document_loaders.nuclia import NucliaLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.nuclia.NucliaLoader']}, 'NVIDIA RIVA': {'imports': 'from langchain_community.utilities.nvidia_riva import RivaASR\nor\nfrom langchain_community.utilities.nvidia_riva import RivaTTS', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.nvidia_riva.RivaASR', 'langchain_community.utilities.nvidia_riva.RivaTTS']}, 'Office365 Toolkit': {'imports': 'from langchain_community.agent_toolkits import O365Toolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.O365Toolkit']}, 'OpenAPI Toolkit': {'imports': 'from langchain_community.agent_toolkits import OpenAPIToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.OpenAPIToolkit']}, 'Natural Language API Toolkit': {'imports': 'from langchain_community.agent_toolkits import NLAToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.NLAToolkit']}, 'OpenWeatherMap': {'imports': 'from langchain_community.utilities import OpenWeatherMapAPIWrapper\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["openweathermap-api"])', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.OpenWeatherMapAPIWrapper', 'langchain.agents.load_tools']}, 'Oracle AI Vector Search Generate Summary': {'imports': 'from langchain_community.utilities.oracleai import OracleSummary', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.oracleai.OracleSummary']}, 'Pandas Dataframe': {'imports': 'from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent', 'category': 'GENERAL', 'import_list': ['langchain_experimental.agents.agent_toolkits.create_pandas_dataframe_agent']}, 'Passio NutritionAI': {'imports': 'from langchain_community.tools.passio_nutrition_ai import NutritionAI\nor\nfrom langchain_community.utilities.passio_nutrition_ai import NutritionAIAPI\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.passio_nutrition_ai.NutritionAI', 'langchain_community.utilities.passio_nutrition_ai.NutritionAIAPI']}, 'PaymanAI': {'imports': 'from langchain_community.tools.langchain_payman_tool.tool import PaymanAI', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.langchain_payman_tool.tool.PaymanAI']}, 'PlayWright Browser Toolkit': {'imports': 'from langchain_community.agent_toolkits import PlayWrightBrowserToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.PlayWrightBrowserToolkit']}, 'Polygon IO Toolkit': {'imports': 'from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit\nor\nfrom langchain_community.utilities.polygon import PolygonAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.polygon.toolkit.PolygonToolkit', 'langchain_community.utilities.polygon.PolygonAPIWrapper']}, 'PowerBI Toolkit': {'imports': 'from langchain_community.agent_toolkits import PowerBIToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.PowerBIToolkit']}, 'Polygon Stock Market API': {'imports': 'from langchain.tools.polygon import PolygonStockMarket', 'category': 'GENERAL', 'import_list': ['langchain.tools.polygon.PolygonStockMarket']}, 'PubMed': {'imports': 'from langchain_community.document_loaders import PubMedLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PubMedLoader']}, 'Python REPL': {'imports': 'from langchain_experimental.utilities import PythonREPL', 'category': 'GENERAL', 'import_list': ['langchain_experimental.utilities.PythonREPL']}, 'Reddit Search': {'imports': 'from langchain_community.tools.reddit_search.tool import RedditSearchRun\nor\nfrom langchain_community.utilities.reddit_search import RedditSearchAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.reddit_search.tool.RedditSearchRun', 'langchain_community.utilities.reddit_search.RedditSearchAPIWrapper']}, 'Requests': {'imports': 'from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.openapi.toolkit.RequestsToolkit']}, 'Riza Code Interpreter': {'imports': 'from langchain_community.tools.riza.command import ExecPython', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.riza.command.ExecPython']}, 'Robocorp Toolkit': {'imports': 'from langchain_robocorp import ActionServerToolkit', 'category': 'GENERAL', 'import_list': ['langchain_robocorp.ActionServerToolkit']}, 'SceneXplain': {'imports': 'from langchain_community.tools import SceneXplainTool\nor\nfrom langchain.agents import load_tools\ntools = load_tools(["sceneXplain"])', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.SceneXplainTool', 'langchain.agents.load_tools']}, 'ScrapeGraph': {'imports': 'from langchain_scrapegraph.tools import GetCreditsTool\nor\nfrom langchain_scrapegraph.tools import LocalScraperTool\nor\nfrom langchain_scrapegraph.tools import MarkdownifyTool\nor\nfrom langchain_scrapegraph.tools import SmartScraperTool', 'category': 'GENERAL', 'import_list': ['langchain_scrapegraph.tools.GetCreditsTool', 'langchain_scrapegraph.tools.LocalScraperTool', 'langchain_scrapegraph.tools.MarkdownifyTool', 'langchain_scrapegraph.tools.SmartScraperTool']}, 'SearchApi': {'imports': 'from langchain_community.utilities import SearchApiAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.SearchApiAPIWrapper']}, 'SearxNG Search': {'imports': 'from langchain_community.utilities import SearxSearchWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.SearxSearchWrapper']}, 'Semantic Scholar API': {'imports': 'from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.semanticscholar.tool.SemanticScholarQueryRun']}, 'SerpAPI': {'imports': 'from langchain_community.utilities import SerpAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.SerpAPIWrapper']}, 'Slack Toolkit': {'imports': 'from langchain_community.agent_toolkits import SlackToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.SlackToolkit']}, 'Spark SQL Toolkit': {'imports': 'from langchain_community.agent_toolkits import SparkSQLToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.SparkSQLToolkit']}, 'SQLDatabase Toolkit': {'imports': 'from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit']}, 'StackExchange': {'imports': 'from langchain_community.utilities import StackExchangeAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.StackExchangeAPIWrapper']}, 'Steam Toolkit': {'imports': 'from langchain_community.agent_toolkits.steam.toolkit import SteamToolkit\nor\nfrom langchain_community.utilities.steam import SteamWebAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.steam.toolkit.SteamToolkit', 'langchain_community.utilities.steam.SteamWebAPIWrapper']}, 'Stripe': {'imports': 'from langchain_community.document_loaders import StripeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.StripeLoader']}, 'Tavily Search': {'imports': 'from langchain_community.tools import TavilySearchResults', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.TavilySearchResults']}, 'Tilores': {'imports': 'from tilores import TiloresAPI\nor\nfrom tilores_langchain import TiloresTools\n', 'category': 'GENERAL', 'import_list': ['tilores.TiloresAPI', 'tilores_langchain.TiloresTools']}, 'Twilio': {'imports': 'from langchain_community.utilities.twilio import TwilioAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.twilio.TwilioAPIWrapper']}, 'Upstage Groundedness Check': {'imports': 'from langchain_upstage import UpstageGroundednessCheck', 'category': 'GENERAL', 'import_list': ['langchain_upstage.UpstageGroundednessCheck']}, 'Wikidata': {'imports': 'from langchain_community.tools.wikidata.tool import WikidataAPIWrapper\nor\nfrom langchain_community.tools.wikidata.tool import WikidataQueryRun\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.wikidata.tool.WikidataAPIWrapper', 'langchain_community.tools.wikidata.tool.WikidataQueryRun']}, 'Wikipedia': {'imports': 'from langchain_community.document_loaders import WikipediaLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.WikipediaLoader']}, 'Wolfram Alpha': {'imports': 'from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper', 'category': 'GENERAL', 'import_list': ['langchain_community.utilities.wolfram_alpha.WolframAlphaAPIWrapper']}, 'Yahoo Finance News': {'imports': 'from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.yahoo_finance_news.YahooFinanceNewsTool']}, 'You.com Search': {'imports': 'from langchain_community.tools.you import YouSearchTool\nor\nfrom langchain_community.utilities.you import YouSearchAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.you.YouSearchTool', 'langchain_community.utilities.you.YouSearchAPIWrapper']}, 'YouTube': {'imports': 'from langchain_community.tools import YouTubeSearchTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.YouTubeSearchTool']}, 'Zapier Natural Language Actions': {'imports': 'from langchain_community.agent_toolkits import ZapierToolkit\nor\nfrom langchain_community.utilities.zapier import ZapierNLAWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.ZapierToolkit', 'langchain_community.utilities.zapier.ZapierNLAWrapper']}, 'ZenGuard AI': {'imports': 'from langchain_community.tools.zenguard import ZenGuardTool', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.zenguard.ZenGuardTool']}, 'AI21 Labs': {'imports': 'from langchain_ai21 import ChatAI21', 'category': 'LLM', 'import_list': ['langchain_ai21.ChatAI21']}, 'Alibaba Cloud PAI EAS': {'imports': 'from langchain_community.chat_models import PaiEasChatEndpoint', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.PaiEasChatEndpoint']}, 'Anthropic': {'imports': 'from langchain_anthropic import ChatAnthropic', 'category': 'LLM', 'import_list': ['langchain_anthropic.ChatAnthropic']}, 'Anyscale': {'imports': 'from langchain_community.chat_models import ChatAnyscale', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatAnyscale']}, 'Azure OpenAI': {'imports': 'from langchain_openai import AzureChatOpenAI', 'category': 'LLM', 'import_list': ['langchain_openai.AzureChatOpenAI']}, 'Azure ML Endpoint': {'imports': 'from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.azureml_endpoint.AzureMLChatOnlineEndpoint']}, 'Baichuan Chat': {'imports': 'from langchain_community.chat_models import ChatBaichuan', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatBaichuan']}, 'Baidu Qianfan': {'imports': 'from langchain_community.chat_models import QianfanChatEndpoint', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.QianfanChatEndpoint']}, 'AWS Bedrock': {'imports': 'from langchain_aws import ChatBedrock', 'category': 'LLM', 'import_list': ['langchain_aws.ChatBedrock']}, 'Cerebras': {'imports': 'from langchain_cerebras import ChatCerebras', 'category': 'LLM', 'import_list': ['langchain_cerebras.ChatCerebras']}, 'Cloudflare Workers AI': {'imports': 'from langchain_community.chat_models.cloudflare_workersai import ChatCloudflareWorkersAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.cloudflare_workersai.ChatCloudflareWorkersAI']}, 'Cohere': {'imports': 'from langchain_cohere import ChatCohere', 'category': 'LLM', 'import_list': ['langchain_cohere.ChatCohere']}, 'Coze Chat': {'imports': 'from langchain_community.chat_models import ChatCoze', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatCoze']}, 'Dappier AI': {'imports': 'from langchain_community.chat_models.dappier import ChatDappierAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.dappier.ChatDappierAI']}, 'Databricks': {'imports': 'from databricks_langchain import ChatDatabricks', 'category': 'LLM', 'import_list': ['databricks_langchain.ChatDatabricks']}, 'DeepInfra': {'imports': 'from langchain_community.chat_models import ChatDeepInfra', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatDeepInfra']}, 'DeepSeek': {'imports': 'from langchain_deepseek import ChatDeepSeek', 'category': 'LLM', 'import_list': ['langchain_deepseek.ChatDeepSeek']}, 'EverlyAI': {'imports': 'from langchain_community.chat_models import ChatEverlyAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatEverlyAI']}, 'Fireworks': {'imports': 'from langchain_fireworks import ChatFireworks', 'category': 'LLM', 'import_list': ['langchain_fireworks.ChatFireworks']}, 'ChatFriendli': {'imports': 'from langchain_community.chat_models.friendli import ChatFriendli', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.friendli.ChatFriendli']}, 'GigaChat': {'imports': 'from langchain_gigachat import GigaChat', 'category': 'LLM', 'import_list': ['langchain_gigachat.GigaChat']}, 'Goodfire': {'imports': 'from langchain_goodfire import ChatGoodfire', 'category': 'LLM', 'import_list': ['langchain_goodfire.ChatGoodfire']}, 'Google AI': {'imports': 'from langchain_google_genai import ChatGoogleGenerativeAI', 'category': 'LLM', 'import_list': ['langchain_google_genai.ChatGoogleGenerativeAI']}, 'Google Cloud Vertex AI': {'imports': 'from langchain_google_vertexai import ChatVertexAI', 'category': 'LLM', 'import_list': ['langchain_google_vertexai.ChatVertexAI']}, 'GPTRouter': {'imports': 'from langchain_community.chat_models import GPTRouter', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.GPTRouter']}, 'Groq': {'imports': 'from langchain_groq import ChatGroq', 'category': 'LLM', 'import_list': ['langchain_groq.ChatGroq']}, 'ChatHuggingFace': {'imports': 'from langchain_huggingface import ChatHuggingFace', 'category': 'LLM', 'import_list': ['langchain_huggingface.ChatHuggingFace']}, 'IBM watsonx.ai': {'imports': 'from langchain_ibm import WatsonxRerank', 'category': 'RETRIEVER', 'import_list': ['langchain_ibm.WatsonxRerank']}, 'JinaChat': {'imports': 'from langchain_community.chat_models import JinaChat', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.JinaChat']}, 'Kinetica': {'imports': 'from langchain_community.document_loaders.kinetica_loader import KineticaLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.kinetica_loader.KineticaLoader']}, 'Konko': {'imports': 'from langchain_community.chat_models import ChatKonko', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatKonko']}, 'LiteLLM': {'imports': 'from langchain_community.chat_models import ChatLiteLLM', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatLiteLLM']}, 'LiteLLMRouter': {'imports': 'from langchain_community.chat_models import ChatLiteLLMRouter', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatLiteLLMRouter']}, 'Llama 2 Chat': {'imports': 'from langchain_experimental.chat_models import Llama2Chat', 'category': 'LLM', 'import_list': ['langchain_experimental.chat_models.Llama2Chat']}, 'Llama API': {'imports': 'from langchain_experimental.llms import ChatLlamaAPI', 'category': 'LLM', 'import_list': ['langchain_experimental.llms.ChatLlamaAPI']}, 'LlamaEdge': {'imports': 'from langchain_community.chat_models.llama_edge import LlamaEdgeChatService', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.llama_edge.LlamaEdgeChatService']}, 'Llama.cpp': {'imports': 'from langchain_community.chat_models import ChatLlamaCpp', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatLlamaCpp']}, 'maritalk': {'imports': 'from langchain_community.chat_models import ChatMaritalk', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatMaritalk']}, 'MiniMax': {'imports': 'from langchain_community.chat_models import MiniMaxChat', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.MiniMaxChat']}, 'MistralAI': {'imports': 'from langchain_mistralai import ChatMistralAI', 'category': 'LLM', 'import_list': ['langchain_mistralai.ChatMistralAI']}, 'MLX': {'imports': 'from langchain_community.chat_models.mlx import ChatMLX', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.mlx.ChatMLX']}, 'ModelScope': {'imports': 'from langchain_modelscope import ModelScopeChatEndpoint', 'category': 'LLM', 'import_list': ['langchain_modelscope.ModelScopeChatEndpoint']}, 'Moonshot': {'imports': 'from langchain_community.chat_models.moonshot import MoonshotChat', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.moonshot.MoonshotChat']}, 'Naver': {'imports': 'from langchain_community.chat_models import ChatClovaX', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatClovaX']}, 'NVIDIA AI Endpoints': {'imports': 'from langchain_nvidia_ai_endpoints import ChatNVIDIA', 'category': 'LLM', 'import_list': ['langchain_nvidia_ai_endpoints.ChatNVIDIA']}, 'ChatOCIModelDeployment': {'imports': 'from langchain_community.chat_models import ChatOCIModelDeployment', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatOCIModelDeployment']}, 'OCIGenAI': {'imports': 'from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.oci_generative_ai.ChatOCIGenAI']}, 'ChatOctoAI': {'imports': 'from langchain_community.chat_models import ChatOctoAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatOctoAI']}, 'Ollama': {'imports': 'from langchain_ollama import ChatOllama', 'category': 'LLM', 'import_list': ['langchain_ollama.ChatOllama']}, 'OpenAI': {'imports': 'from langchain_openai import ChatOpenAI', 'category': 'LLM', 'import_list': ['langchain_openai.ChatOpenAI']}, 'Outlines': {'imports': 'from langchain_community.chat_models.outlines import ChatOutlines', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.outlines.ChatOutlines']}, 'Perplexity': {'imports': 'from langchain_community.chat_models import ChatPerplexity', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatPerplexity']}, 'Pipeshift': {'imports': 'from langchain_pipeshift import ChatPipeshift', 'category': 'LLM', 'import_list': ['langchain_pipeshift.ChatPipeshift']}, 'ChatPredictionGuard': {'imports': 'from langchain_predictionguard import ChatPredictionGuard', 'category': 'LLM', 'import_list': ['langchain_predictionguard.ChatPredictionGuard']}, 'PremAI': {'imports': 'from langchain_community.chat_models import ChatPremAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatPremAI']}, 'PromptLayer ChatOpenAI': {'imports': 'from langchain_community.chat_models import PromptLayerChatOpenAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.PromptLayerChatOpenAI']}, 'Reka': {'imports': 'from langchain_community.chat_models import ChatReka', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatReka']}, 'SambaNovaCloud': {'imports': 'from langchain_sambanova import ChatSambaNovaCloud', 'category': 'LLM', 'import_list': ['langchain_sambanova.ChatSambaNovaCloud']}, 'SambaStudio': {'imports': 'from langchain_sambanova import ChatSambaStudio', 'category': 'LLM', 'import_list': ['langchain_sambanova.ChatSambaStudio']}, 'Snowflake Cortex': {'imports': 'from langchain_community.chat_models import ChatSnowflakeCortex', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatSnowflakeCortex']}, 'solar': {'imports': 'from langchain_community.chat_models.solar import SolarChat', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.solar.SolarChat']}, 'SparkLLM Chat': {'imports': 'from langchain_community.chat_models import ChatSparkLLM', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatSparkLLM']}, 'Nebula (Symbl.ai)': {'imports': 'from langchain_community.chat_models.symblai_nebula import ChatNebula', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.symblai_nebula.ChatNebula']}, 'Tencent Hunyuan': {'imports': 'from langchain_community.chat_models import ChatHunyuan', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatHunyuan']}, 'Together': {'imports': 'from langchain_together import ChatTogether', 'category': 'LLM', 'import_list': ['langchain_together.ChatTogether']}, 'Tongyi Qwen': {'imports': 'from langchain_community.chat_models.tongyi import ChatTongyi', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.tongyi.ChatTongyi']}, 'Upstage': {'imports': 'from langchain_upstage import UpstageDocumentParseLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_upstage.UpstageDocumentParseLoader']}, 'Volc Enging Maas': {'imports': 'from langchain_community.chat_models import VolcEngineMaasChat', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.VolcEngineMaasChat']}, 'Writer': {'imports': 'from langchain_community.chat_models.writer import ChatWriter', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.writer.ChatWriter']}, 'xAI': {'imports': 'from langchain_xai import ChatXAI', 'category': 'LLM', 'import_list': ['langchain_xai.ChatXAI']}, 'YandexGPT': {'imports': 'from langchain_community.chat_models import ChatYandexGPT', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatYandexGPT']}, 'ChatYI': {'imports': 'from langchain_community.chat_models.yi import ChatYi', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.yi.ChatYi']}, 'Yuan2.0': {'imports': 'from langchain_community.chat_models import ChatYuan2', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatYuan2']}, 'ZHIPU AI': {'imports': 'from langchain_community.chat_models import ChatZhipuAI', 'category': 'LLM', 'import_list': ['langchain_community.chat_models.ChatZhipuAI']}, 'Amazon Kendra': {'imports': 'from langchain_community.retrievers import AmazonKendraRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.AmazonKendraRetriever']}, 'Arcee': {'imports': 'from langchain_community.retrievers import ArceeRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.ArceeRetriever']}, 'Arxiv': {'imports': 'from langchain_community.retrievers import ArxivRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.ArxivRetriever']}, 'Azure AI Search': {'imports': 'from langchain_community.vectorstores.azuresearch import AzureSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.azuresearch.AzureSearch']}, 'Bedrock (Knowledge Bases)': {'imports': 'from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_aws.retrievers.AmazonKnowledgeBasesRetriever']}, 'BM25': {'imports': 'from langchain_community.retrievers import BM25Retriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.BM25Retriever']}, 'Box': {'imports': 'from langchain_box.blob_loaders import BoxBlobLoader\nor\nfrom langchain_box.document_loaders import BoxLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_box.blob_loaders.BoxBlobLoader', 'langchain_box.document_loaders.BoxLoader']}, 'BREEBS (Open Knowledge)': {'imports': 'from langchain_community.retrievers import BreebsRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.BreebsRetriever']}, 'Chaindesk': {'imports': 'from langchain_community.retrievers import ChaindeskRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.ChaindeskRetriever']}, 'ChatGPT plugin': {'imports': 'from langchain_community.retrievers import ChatGPTPluginRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.ChatGPTPluginRetriever']}, 'Cohere reranker': {'imports': 'from langchain_cohere import CohereRerank', 'category': 'RETRIEVER', 'import_list': ['langchain_cohere.CohereRerank']}, 'Cohere RAG': {'imports': 'from langchain_cohere import CohereRagRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_cohere.CohereRagRetriever']}, 'DocArray': {'imports': 'from langchain_community.retrievers import DocArrayRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.DocArrayRetriever']}, 'Dria': {'imports': 'from langchain_community.retrievers import DriaRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.DriaRetriever']}, 'ElasticSearch BM25': {'imports': 'from langchain_community.retrievers import ElasticSearchBM25Retriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.ElasticSearchBM25Retriever']}, 'ElasticsearchRetriever': {'imports': 'from langchain_elasticsearch import ElasticsearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_elasticsearch.ElasticsearchRetriever']}, 'Embedchain': {'imports': 'from langchain_community.retrievers import EmbedchainRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.EmbedchainRetriever']}, 'FlashRank reranker': {'imports': 'from langchain.retrievers.document_compressors import FlashrankRerank', 'category': 'RETRIEVER', 'import_list': ['langchain.retrievers.document_compressors.FlashrankRerank']}, 'Fleet AI Context': {'imports': 'from context import download_embeddings', 'category': 'RETRIEVER', 'import_list': ['context.download_embeddings']}, 'Google Vertex AI Search': {'imports': 'from langchain_google_community import VertexAIMultiTurnSearchRetriever\nor\nfrom langchain_google_community import VertexAISearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_google_community.VertexAIMultiTurnSearchRetriever', 'langchain_google_community.VertexAISearchRetriever']}, 'JaguarDB Vector Database': {'imports': 'from langchain_community.vectorstores.jaguar import Jaguar', 'category': 'RETRIEVER', 'import_list': ['langchain_community.vectorstores.jaguar.Jaguar']}, 'Kay.ai': {'imports': 'from langchain_community.retrievers import KayAiRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.KayAiRetriever']}, 'Kinetica Vectorstore based Retriever': {'imports': 'from langchain_community.vectorstores import Kinetica', 'category': 'RETRIEVER', 'import_list': ['langchain_community.vectorstores.Kinetica']}, 'kNN': {'imports': 'from langchain_community.retrievers import KNNRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.KNNRetriever']}, 'LinkupSearchRetriever': {'imports': 'from langchain_linkup import LinkupSearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_linkup.LinkupSearchRetriever']}, 'LLMLingua Document Compressor': {'imports': 'from langchain_community.document_compressors import LLMLinguaCompressor', 'category': 'RETRIEVER', 'import_list': ['langchain_community.document_compressors.LLMLinguaCompressor']}, 'LOTR (Merger Retriever)': {'imports': 'from langchain.retrievers import MergerRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain.retrievers.MergerRetriever']}, 'Metal': {'imports': 'from langchain_community.retrievers import MetalRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.MetalRetriever']}, 'Milvus Hybrid Search': {'imports': 'from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_milvus.retrievers.MilvusCollectionHybridSearchRetriever']}, 'NanoPQ (Product Quantization)': {'imports': 'from langchain_community.retrievers import NanoPQRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.NanoPQRetriever']}, 'needle': {'imports': 'from langchain_community.retrievers.needle import NeedleRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.needle.NeedleRetriever']}, 'Nimble': {'imports': 'from langchain_nimble import NimbleSearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_nimble.NimbleSearchRetriever']}, 'Outline': {'imports': 'from langchain_community.retrievers import OutlineRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.OutlineRetriever']}, 'Pinecone Hybrid Search': {'imports': 'from langchain_community.retrievers import PineconeHybridSearchRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.PineconeHybridSearchRetriever']}, 'Qdrant Sparse Vector': {'imports': 'from langchain_community.retrievers import QdrantSparseVectorRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.QdrantSparseVectorRetriever']}, 'RAGatouille': {'imports': 'from ragatouille import RAGPretrainedModel', 'category': 'RETRIEVER', 'import_list': ['ragatouille.RAGPretrainedModel']}, 'RePhraseQuery': {'imports': 'from langchain.retrievers import RePhraseQueryRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain.retrievers.RePhraseQueryRetriever']}, 'Rememberizer': {'imports': 'from langchain_community.retrievers import RememberizerRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.RememberizerRetriever']}, 'Self-querying retriever': {'imports': 'from langchain.retrievers.self_query.base import SelfQueryRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain.retrievers.self_query.base.SelfQueryRetriever']}, 'SingleStoreDB': {'imports': 'from langchain_community.vectorstores import SingleStoreDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SingleStoreDB']}, 'SVM': {'imports': 'from langchain_community.retrievers import SVMRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.SVMRetriever']}, 'TavilySearchAPI': {'imports': 'from langchain_community.retrievers import TavilySearchAPIRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.TavilySearchAPIRetriever']}, 'TF-IDF': {'imports': 'from langchain_community.retrievers import TFIDFRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.TFIDFRetriever']}, 'NeuralDB': {'imports': 'from langchain_community.retrievers import NeuralDBRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.NeuralDBRetriever']}, 'Vespa': {'imports': 'from langchain_community.vectorstores import VespaStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.VespaStore']}, 'You.com': {'imports': 'from langchain_community.retrievers.you import YouRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.you.YouRetriever']}, 'Zep Cloud': {'imports': 'from langchain_community.vectorstores import ZepCloudVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.ZepCloudVectorStore']}, 'Zep Open Source': {'imports': 'from langchain_community.retrievers.zep import ZepRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_community.retrievers.zep.ZepRetriever']}, 'Zilliz Cloud Pipeline': {'imports': 'from langchain_milvus import ZillizCloudPipelineRetriever', 'category': 'RETRIEVER', 'import_list': ['langchain_milvus.ZillizCloudPipelineRetriever']}, 'acreom': {'imports': 'from langchain_community.document_loaders import AcreomLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AcreomLoader']}, 'AirbyteLoader': {'imports': 'from langchain_airbyte import AirbyteLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_airbyte.AirbyteLoader']}, 'Airtable': {'imports': 'from langchain_community.document_loaders import AirtableLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AirtableLoader']}, 'Alibaba Cloud MaxCompute': {'imports': 'from langchain_community.document_loaders import MaxComputeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.MaxComputeLoader']}, 'Amazon Textract': {'imports': 'from langchain_community.document_loaders import AmazonTextractPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AmazonTextractPDFLoader']}, 'Apify Dataset': {'imports': 'from langchain_community.document_loaders import ApifyDatasetLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ApifyDatasetLoader']}, 'ArcGIS': {'imports': 'from langchain_community.document_loaders import ArcGISLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ArcGISLoader']}, 'ArxivLoader': {'imports': 'from langchain_community.document_loaders import ArxivLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ArxivLoader']}, 'AssemblyAI Audio Transcripts': {'imports': 'from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AssemblyAIAudioTranscriptLoader']}, 'AstraDB': {'imports': 'from langchain_community.document_loaders import AstraDBLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AstraDBLoader']}, 'Async Chromium': {'imports': 'from langchain_community.document_loaders import AsyncChromiumLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AsyncChromiumLoader']}, 'AsyncHtml': {'imports': 'from langchain_community.document_loaders import AsyncHtmlLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AsyncHtmlLoader']}, 'Athena': {'imports': 'from langchain_community.document_loaders.athena import AthenaLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.athena.AthenaLoader']}, 'AWS S3 Directory': {'imports': 'from langchain_community.document_loaders import S3DirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.S3DirectoryLoader']}, 'AWS S3 File': {'imports': 'from langchain_community.document_loaders import S3FileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.S3FileLoader']}, 'AZLyrics': {'imports': 'from langchain_community.document_loaders import AZLyricsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AZLyricsLoader']}, 'Azure AI Data': {'imports': 'from langchain_community.document_loaders import AzureAIDataLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AzureAIDataLoader']}, 'Azure Blob Storage Container': {'imports': 'from langchain_community.document_loaders import AzureBlobStorageContainerLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AzureBlobStorageContainerLoader']}, 'Azure Blob Storage File': {'imports': 'from langchain_community.document_loaders import AzureBlobStorageFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AzureBlobStorageFileLoader']}, 'Azure AI Document Intelligence': {'imports': 'from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.AzureAIDocumentIntelligenceLoader']}, 'BibTeX': {'imports': 'from langchain_community.document_loaders import BibtexLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BibtexLoader']}, 'BiliBili': {'imports': 'from langchain_community.document_loaders import BiliBiliLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BiliBiliLoader']}, 'Blackboard': {'imports': 'from langchain_community.document_loaders import BlackboardLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BlackboardLoader']}, 'Blockchain': {'imports': 'from langchain_community.document_loaders.blockchain import BlockchainDocumentLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.blockchain.BlockchainDocumentLoader']}, 'Browserbase': {'imports': 'from langchain_community.document_loaders import BrowserbaseLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BrowserbaseLoader']}, 'Browserless': {'imports': 'from langchain_community.document_loaders import BrowserlessLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BrowserlessLoader']}, 'BSHTMLLoader': {'imports': 'from langchain_community.document_loaders import BSHTMLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.BSHTMLLoader']}, 'Cassandra': {'imports': 'from langchain_community.document_loaders import CassandraLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.CassandraLoader']}, 'ChatGPT Data': {'imports': 'from langchain_community.document_loaders.chatgpt import ChatGPTLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.chatgpt.ChatGPTLoader']}, 'College Confidential': {'imports': 'from langchain_community.document_loaders import CollegeConfidentialLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.CollegeConfidentialLoader']}, 'Concurrent Loader': {'imports': 'from langchain_community.document_loaders import ConcurrentLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ConcurrentLoader']}, 'Confluence': {'imports': 'from langchain_community.document_loaders import ConfluenceLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ConfluenceLoader']}, 'CoNLL-U': {'imports': 'from langchain_community.document_loaders import CoNLLULoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.CoNLLULoader']}, 'Couchbase': {'imports': 'from langchain_couchbase.vectorstores import CouchbaseVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_couchbase.vectorstores.CouchbaseVectorStore']}, 'CSV': {'imports': 'from langchain_community.document_loaders.csv_loader import CSVLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.csv_loader.CSVLoader']}, 'Cube Semantic Layer': {'imports': 'from langchain_community.document_loaders import CubeSemanticLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.CubeSemanticLoader']}, 'Datadog Logs': {'imports': 'from langchain_community.document_loaders import DatadogLogsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DatadogLogsLoader']}, 'Dedoc': {'imports': 'from langchain_community.document_loaders import DedocFileLoader\nor\nfrom langchain_community.document_loaders import DedocPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DedocFileLoader', 'langchain_community.document_loaders.DedocPDFLoader']}, 'Diffbot': {'imports': 'from langchain_community.document_loaders import DiffbotLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DiffbotLoader']}, 'Discord': {'imports': 'from langchain_community.document_loaders.discord import DiscordChatLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.discord.DiscordChatLoader']}, 'Docling': {'imports': 'from langchain_docling import DoclingLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_docling.DoclingLoader']}, 'Docugami': {'imports': 'from docugami_langchain.document_loaders import DocugamiLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['docugami_langchain.document_loaders.DocugamiLoader']}, 'Docusaurus': {'imports': 'from langchain_community.document_loaders import DocusaurusLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DocusaurusLoader']}, 'Dropbox': {'imports': 'from langchain_community.document_loaders import DropboxLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DropboxLoader']}, 'DuckDB': {'imports': 'from langchain_community.vectorstores import DuckDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.DuckDB']}, 'Email': {'imports': 'from langchain_community.document_loaders import UnstructuredEmailLoader\nor\nfrom langchain_community.document_loaders import OutlookMessageLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredEmailLoader', 'langchain_community.document_loaders.OutlookMessageLoader']}, 'EPub': {'imports': 'from langchain_community.document_loaders import UnstructuredEPubLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredEPubLoader']}, 'Etherscan': {'imports': 'from langchain_community.document_loaders import EtherscanLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.EtherscanLoader']}, 'EverNote': {'imports': 'from langchain_community.document_loaders import EverNoteLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.EverNoteLoader']}, 'Facebook Chat': {'imports': 'from langchain_community.document_loaders import FacebookChatLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.FacebookChatLoader']}, 'Fauna': {'imports': 'from langchain_community.document_loaders.fauna import FaunaLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.fauna.FaunaLoader']}, 'Figma': {'imports': 'from langchain_community.document_loaders.figma import FigmaFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.figma.FigmaFileLoader']}, 'FireCrawl': {'imports': 'from langchain_community.document_loaders.firecrawl import FireCrawlLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.firecrawl.FireCrawlLoader']}, 'Geopandas': {'imports': 'from langchain_community.document_loaders import GeoDataFrameLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.GeoDataFrameLoader']}, 'Git': {'imports': 'from langchain_community.document_loaders import GitLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.GitLoader']}, 'GitBook': {'imports': 'from langchain_community.document_loaders import GitbookLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.GitbookLoader']}, 'GitHub': {'imports': 'from langchain_community.document_loaders import GitHubIssuesLoader\nor\nfrom langchain_community.document_loaders import GithubFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.GitHubIssuesLoader', 'langchain_community.document_loaders.GithubFileLoader']}, 'Glue Catalog': {'imports': 'from langchain_community.document_loaders.glue_catalog import GlueCatalogLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.glue_catalog.GlueCatalogLoader']}, 'Google AlloyDB for PostgreSQL': {'imports': 'from langchain_google_alloydb_pg import AlloyDBVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_alloydb_pg.AlloyDBVectorStore']}, 'Google BigQuery': {'imports': 'from langchain_google_community import BigQueryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_community.BigQueryLoader']}, 'Google Bigtable': {'imports': 'from langchain_google_bigtable import BigtableLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_bigtable.BigtableLoader']}, 'Google Cloud SQL for SQL server': {'imports': 'from langchain_google_cloud_sql_mssql import MSSQLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_cloud_sql_mssql.MSSQLLoader']}, 'Google Cloud SQL for MySQL': {'imports': 'from langchain_google_cloud_sql_mysql import MySQLVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_cloud_sql_mysql.MySQLVectorStore']}, 'Google Cloud SQL for PostgreSQL': {'imports': 'from langchain_google_cloud_sql_pg import PostgresVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_cloud_sql_pg.PostgresVectorStore']}, 'Google Cloud Storage Directory': {'imports': 'from langchain_google_community import GCSDirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_community.GCSDirectoryLoader']}, 'Google Cloud Storage File': {'imports': 'from langchain_google_community import GCSFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_community.GCSFileLoader']}, 'Google Firestore in Datastore Mode': {'imports': 'from langchain_google_datastore import DatastoreLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_datastore.DatastoreLoader']}, 'Google El Carro for Oracle Workloads': {'imports': 'from langchain_google_el_carro import ElCarroLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_el_carro.ElCarroLoader']}, 'Google Firestore (Native Mode)': {'imports': 'from langchain_google_firestore import FirestoreLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_firestore.FirestoreLoader']}, 'Google Memorystore for Redis': {'imports': 'from langchain_google_memorystore_redis import RedisVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_memorystore_redis.RedisVectorStore']}, 'Google Spanner': {'imports': 'from langchain_google_spanner import SpannerVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_spanner.SpannerVectorStore']}, 'Google Speech-to-Text Audio Transcripts': {'imports': 'from langchain_google_community import SpeechToTextLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_google_community.SpeechToTextLoader']}, 'Grobid': {'imports': 'from langchain_community.document_loaders.generic import GenericLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.generic.GenericLoader']}, 'Gutenberg': {'imports': 'from langchain_community.document_loaders import GutenbergLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.GutenbergLoader']}, 'Hacker News': {'imports': 'from langchain_community.document_loaders import HNLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.HNLoader']}, 'Huawei OBS Directory': {'imports': 'from langchain_community.document_loaders import OBSDirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.OBSDirectoryLoader']}, 'Huawei OBS File': {'imports': 'from langchain_community.document_loaders.obs_file import OBSFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.obs_file.OBSFileLoader']}, 'HuggingFace dataset': {'imports': 'from langchain_community.document_loaders import HuggingFaceDatasetLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.HuggingFaceDatasetLoader']}, 'HyperbrowserLoader': {'imports': 'from langchain_hyperbrowser import HyperbrowserLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_hyperbrowser.HyperbrowserLoader']}, 'iFixit': {'imports': 'from langchain_community.document_loaders import IFixitLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.IFixitLoader']}, 'Images': {'imports': 'from langchain_community.document_loaders.image import UnstructuredImageLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.image.UnstructuredImageLoader']}, 'Image captions': {'imports': 'from langchain_community.document_loaders import ImageCaptionLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ImageCaptionLoader']}, 'IMSDb': {'imports': 'from langchain_community.document_loaders import IMSDbLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.IMSDbLoader']}, 'Iugu': {'imports': 'from langchain_community.document_loaders import IuguLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.IuguLoader']}, 'Joplin': {'imports': 'from langchain_community.document_loaders import JoplinLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.JoplinLoader']}, 'JSONLoader': {'imports': 'from langchain_community.document_loaders import JSONLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.JSONLoader']}, 'Jupyter Notebook': {'imports': 'from langchain_community.document_loaders import NotebookLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.NotebookLoader']}, 'lakeFS': {'imports': 'from langchain_community.document_loaders import LakeFSLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.LakeFSLoader']}, 'LangSmithLoader': {'imports': 'from langchain_core.document_loaders import LangSmithLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_core.document_loaders.LangSmithLoader']}, 'LarkSuite (FeiShu)': {'imports': 'from langchain_community.document_loaders.larksuite import LarkSuiteDocLoader\nor\nfrom langchain_community.document_loaders.larksuite import LarkSuiteWikiLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.larksuite.LarkSuiteDocLoader', 'langchain_community.document_loaders.larksuite.LarkSuiteWikiLoader']}, 'LLM Sherpa': {'imports': 'from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.llmsherpa.LLMSherpaFileLoader']}, 'Mastodon': {'imports': 'from langchain_community.document_loaders import MastodonTootsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.MastodonTootsLoader']}, 'MathPixPDFLoader': {'imports': 'from langchain_community.document_loaders import MathpixPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.MathpixPDFLoader']}, 'MediaWiki Dump': {'imports': 'from langchain_community.document_loaders import MWDumpLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.MWDumpLoader']}, 'Merge Documents Loader': {'imports': 'from langchain_community.document_loaders.merge import MergedDataLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.merge.MergedDataLoader']}, 'mhtml': {'imports': 'from langchain_community.document_loaders import MHTMLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.MHTMLLoader']}, 'Microsoft Excel': {'imports': 'from langchain_community.document_loaders import UnstructuredExcelLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredExcelLoader']}, 'Microsoft OneDrive': {'imports': 'from langchain_community.document_loaders.onedrive import OneDriveLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.onedrive.OneDriveLoader']}, 'Microsoft OneNote': {'imports': 'from langchain_community.document_loaders.onenote import OneNoteLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.onenote.OneNoteLoader']}, 'Microsoft PowerPoint': {'imports': 'from langchain_community.document_loaders import UnstructuredPowerPointLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredPowerPointLoader']}, 'Microsoft SharePoint': {'imports': 'from langchain_community.document_loaders.sharepoint import SharePointLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.sharepoint.SharePointLoader']}, 'Microsoft Word': {'imports': 'from langchain_community.document_loaders import UnstructuredWordDocumentLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredWordDocumentLoader']}, 'Near Blockchain': {'imports': 'from MintbaseLoader import MintbaseDocumentLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['MintbaseLoader.MintbaseDocumentLoader']}, 'Modern Treasury': {'imports': 'from langchain_community.document_loaders import ModernTreasuryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ModernTreasuryLoader']}, 'MongoDB': {'imports': 'from langchain_community.document_loaders.mongodb import MongodbLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.mongodb.MongodbLoader']}, 'Needle Document Loader': {'imports': 'from langchain_community.document_loaders.needle import NeedleLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.needle.NeedleLoader']}, 'News URL': {'imports': 'from langchain_community.document_loaders import NewsURLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.NewsURLLoader']}, 'Notion DB 2/2': {'imports': 'from langchain_community.document_loaders import NotionDBLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.NotionDBLoader']}, 'Obsidian': {'imports': 'from langchain_community.document_loaders import ObsidianLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ObsidianLoader']}, 'Open Document Format (ODT)': {'imports': 'from langchain_community.document_loaders import UnstructuredODTLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredODTLoader']}, 'Open City Data': {'imports': 'from langchain_community.document_loaders import OpenCityDataLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.OpenCityDataLoader']}, 'Oracle Autonomous Database': {'imports': 'from langchain_community.document_loaders import OracleAutonomousDatabaseLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.OracleAutonomousDatabaseLoader']}, 'Oracle AI Vector Search: Document Processing': {'imports': 'from langchain_community.document_loaders.oracleai import OracleDocLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.oracleai.OracleDocLoader']}, 'Org-mode': {'imports': 'from langchain_community.document_loaders import UnstructuredOrgModeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredOrgModeLoader']}, 'Pandas DataFrame': {'imports': 'from langchain_community.document_loaders import DataFrameLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.DataFrameLoader']}, 'PDFMinerLoader': {'imports': 'from langchain_community.document_loaders import PDFMinerLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PDFMinerLoader']}, 'PDFPlumber': {'imports': 'from langchain_community.document_loaders import PDFPlumberLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PDFPlumberLoader']}, 'Pebblo Safe DocumentLoader': {'imports': 'from langchain_community.document_loaders import PebbloSafeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PebbloSafeLoader']}, 'Polars DataFrame': {'imports': 'from langchain_community.document_loaders import PolarsDataFrameLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PolarsDataFrameLoader']}, 'Psychic': {'imports': 'from langchain_community.document_loaders import PsychicLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PsychicLoader']}, 'PullMdLoader': {'imports': 'from langchain_pull_md.markdown_loader import PullMdLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_pull_md.markdown_loader.PullMdLoader']}, 'PyMuPDFLoader': {'imports': 'from langchain_community.document_loaders import PyMuPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PyMuPDFLoader']}, 'PyPDFDirectoryLoader': {'imports': 'from langchain_community.document_loaders import PyPDFDirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PyPDFDirectoryLoader']}, 'PyPDFium2Loader': {'imports': 'from langchain_community.document_loaders import PyPDFium2Loader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PyPDFium2Loader']}, 'PyPDFLoader': {'imports': 'from langchain_community.document_loaders import PyPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PyPDFLoader']}, 'PySpark': {'imports': 'from langchain_community.document_loaders import PySparkDataFrameLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.PySparkDataFrameLoader']}, 'Quip': {'imports': 'from langchain_community.document_loaders.quip import QuipLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.quip.QuipLoader']}, 'ReadTheDocs Documentation': {'imports': 'from langchain_community.document_loaders import ReadTheDocsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ReadTheDocsLoader']}, 'Recursive URL': {'imports': 'from langchain_community.document_loaders import RecursiveUrlLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.RecursiveUrlLoader']}, 'Reddit': {'imports': 'from langchain_community.document_loaders import RedditPostsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.RedditPostsLoader']}, 'Roam': {'imports': 'from langchain_community.document_loaders import RoamLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.RoamLoader']}, 'Rockset': {'imports': 'from langchain_community.vectorstores import Rockset', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Rockset']}, 'rspace': {'imports': 'from langchain_community.document_loaders.rspace import RSpaceLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.rspace.RSpaceLoader']}, 'RSS Feeds': {'imports': 'from langchain_community.document_loaders import RSSFeedLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.RSSFeedLoader']}, 'RST': {'imports': 'from langchain_community.document_loaders import UnstructuredRSTLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredRSTLoader']}, 'scrapfly': {'imports': 'from langchain_community.document_loaders import ScrapflyLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ScrapflyLoader']}, 'ScrapingAnt': {'imports': 'from langchain_community.document_loaders import ScrapingAntLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ScrapingAntLoader']}, 'Sitemap': {'imports': 'from langchain_community.document_loaders.sitemap import SitemapLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.sitemap.SitemapLoader']}, 'Slack': {'imports': 'from langchain_community.document_loaders import SlackDirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.SlackDirectoryLoader']}, 'Snowflake': {'imports': 'from langchain_community.document_loaders import SnowflakeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.SnowflakeLoader']}, 'Spider': {'imports': 'from langchain_community.document_loaders import SpiderLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.SpiderLoader']}, 'Spreedly': {'imports': 'from langchain_community.document_loaders import SpreedlyLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.SpreedlyLoader']}, 'Subtitle': {'imports': 'from langchain_community.document_loaders import SRTLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.SRTLoader']}, 'SurrealDB': {'imports': 'from langchain_community.vectorstores import SurrealDBStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SurrealDBStore']}, 'Telegram': {'imports': 'from langchain_community.document_loaders import TelegramChatApiLoader\nor\nfrom langchain_community.document_loaders import TelegramChatFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TelegramChatApiLoader', 'langchain_community.document_loaders.TelegramChatFileLoader']}, 'Tencent COS Directory': {'imports': 'from langchain_community.document_loaders import TencentCOSDirectoryLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TencentCOSDirectoryLoader']}, 'Tencent COS File': {'imports': 'from langchain_community.document_loaders import TencentCOSFileLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TencentCOSFileLoader']}, 'TensorFlow Datasets': {'imports': 'from langchain_community.document_loaders import TensorflowDatasetLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TensorflowDatasetLoader']}, 'TiDB': {'imports': 'from langchain_community.document_loaders import TiDBLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TiDBLoader']}, '2Markdown': {'imports': 'from langchain_community.document_loaders import ToMarkdownLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.ToMarkdownLoader']}, 'TOML': {'imports': 'from langchain_community.document_loaders import TomlLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TomlLoader']}, 'Trello': {'imports': 'from langchain_community.document_loaders import TrelloLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TrelloLoader']}, 'TSV': {'imports': 'from langchain_community.document_loaders.tsv import UnstructuredTSVLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.tsv.UnstructuredTSVLoader']}, 'Twitter': {'imports': 'from langchain_community.document_loaders import TwitterTweetLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.TwitterTweetLoader']}, 'Unstructured': {'imports': 'from langchain_unstructured import UnstructuredLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_unstructured.UnstructuredLoader']}, 'UnstructuredMarkdownLoader': {'imports': 'from langchain_community.document_loaders import UnstructuredMarkdownLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredMarkdownLoader']}, 'UnstructuredPDFLoader': {'imports': 'from langchain_community.document_loaders import UnstructuredPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredPDFLoader']}, 'URL': {'imports': 'from langchain_community.document_loaders import UnstructuredURLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredURLLoader']}, 'Vsdx': {'imports': 'from langchain_community.document_loaders import VsdxLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.VsdxLoader']}, 'Weather': {'imports': 'from langchain_community.document_loaders import WeatherDataLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.WeatherDataLoader']}, 'WebBaseLoader': {'imports': 'from langchain_community.document_loaders import WebBaseLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.WebBaseLoader']}, 'WhatsApp Chat': {'imports': 'from langchain_community.document_loaders import WhatsAppChatLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.WhatsAppChatLoader']}, 'UnstructuredXMLLoader': {'imports': 'from langchain_community.document_loaders import UnstructuredXMLLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.UnstructuredXMLLoader']}, 'Xorbits Pandas DataFrame': {'imports': 'from langchain_community.document_loaders import XorbitsLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.XorbitsLoader']}, 'YouTube audio': {'imports': 'from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.blob_loaders.youtube_audio.YoutubeAudioLoader']}, 'YouTube transcripts': {'imports': 'from langchain_community.document_loaders import YoutubeLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.YoutubeLoader']}, 'YoutubeLoaderDL': {'imports': 'from langchain_yt_dlp.youtube_loader import YoutubeLoaderDL', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_yt_dlp.youtube_loader.YoutubeLoaderDL']}, 'Yuque': {'imports': 'from langchain_community.document_loaders import YuqueLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.YuqueLoader']}, 'ZeroxPDFLoader': {'imports': 'from langchain_community.document_loaders.pdf import ZeroxPDFLoader', 'category': 'DOCUMENT_LOADER', 'import_list': ['langchain_community.document_loaders.pdf.ZeroxPDFLoader']}, 'Activeloop Deep Lake': {'imports': 'from langchain_community.vectorstores import DeepLake', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.DeepLake']}, 'Aerospike': {'imports': 'from langchain_community.vectorstores import Aerospike', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Aerospike']}, 'Alibaba Cloud OpenSearch': {'imports': 'from langchain_community.vectorstores import AlibabaCloudOpenSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.AlibabaCloudOpenSearch']}, 'AnalyticDB': {'imports': 'from langchain_community.vectorstores import AnalyticDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.AnalyticDB']}, 'Annoy': {'imports': 'from langchain_community.vectorstores import Annoy', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Annoy']}, 'Apache Doris': {'imports': 'from langchain_community.vectorstores.apache_doris import ApacheDoris', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.apache_doris.ApacheDoris']}, 'ApertureDB': {'imports': 'from langchain_community.vectorstores import ApertureDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.ApertureDB']}, 'Astra DB Vector Store': {'imports': 'from langchain_astradb import AstraDBVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_astradb.AstraDBVectorStore']}, 'Atlas': {'imports': 'from langchain_community.vectorstores import AtlasDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.AtlasDB']}, 'AwaDB': {'imports': 'from langchain_community.vectorstores import AwaDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.AwaDB']}, 'Azure Cosmos DB Mongo vCore': {'imports': 'from langchain_community.vectorstores.azure_cosmos_db import AzureCosmosDBVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.azure_cosmos_db.AzureCosmosDBVectorSearch']}, 'Azure Cosmos DB No SQL': {'imports': 'from langchain_community.vectorstores.azure_cosmos_db_no_sql import AzureCosmosDBNoSqlVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.azure_cosmos_db_no_sql.AzureCosmosDBNoSqlVectorSearch']}, 'Bagel': {'imports': 'from langchain_community.vectorstores import Bagel', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Bagel']}, 'BagelDB': {'imports': 'from langchain_community.vectorstores import Bagel', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Bagel']}, 'Baidu Cloud ElasticSearch VectorSearch': {'imports': 'from langchain_community.vectorstores import BESVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.BESVectorStore']}, 'Apache Cassandra': {'imports': 'from langchain_community.vectorstores import Cassandra', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Cassandra']}, 'Chroma': {'imports': 'from langchain_chroma import Chroma', 'category': 'VECTOR_STORE', 'import_list': ['langchain_chroma.Chroma']}, 'Clarifai': {'imports': 'from langchain_community.vectorstores import Clarifai', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Clarifai']}, 'ClickHouse': {'imports': 'from langchain_community.vectorstores import Clickhouse', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Clickhouse']}, 'DashVector': {'imports': 'from langchain_community.vectorstores import DashVector', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.DashVector']}, 'DatabricksVectorSearch': {'imports': 'from databricks_langchain import DatabricksVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['databricks_langchain.DatabricksVectorSearch']}, 'DingoDB': {'imports': 'from langchain_community.vectorstores import Dingo', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Dingo']}, 'DocArray HnswSearch': {'imports': 'from langchain_community.vectorstores import DocArrayHnswSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.DocArrayHnswSearch']}, 'DocArray InMemorySearch': {'imports': 'from langchain_community.vectorstores import DocArrayInMemorySearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.DocArrayInMemorySearch']}, 'Amazon Document DB': {'imports': 'from langchain.vectorstores.documentdb import DocumentDBVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain.vectorstores.documentdb.DocumentDBVectorSearch']}, 'China Mobile ECloud ElasticSearch VectorSearch': {'imports': 'from langchain_community.vectorstores import EcloudESVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.EcloudESVectorStore']}, 'Elasticsearch': {'imports': 'from langchain_elasticsearch import ElasticsearchStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_elasticsearch.ElasticsearchStore']}, 'Epsilla': {'imports': 'from langchain_community.vectorstores import Epsilla', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Epsilla']}, 'Faiss': {'imports': 'from langchain_community.vectorstores import FAISS', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.FAISS']}, 'FalkorDBVectorStore': {'imports': 'from langchain_community.vectorstores.falkordb_vector import FalkorDBVector', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.falkordb_vector.FalkorDBVector']}, 'Google BigQuery Vector Search': {'imports': 'from langchain_google_community import BigQueryVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_community.BigQueryVectorStore']}, 'Google Firestore': {'imports': 'from langchain_google_firestore import FirestoreVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_firestore.FirestoreVectorStore']}, 'Google Vertex AI Feature Store': {'imports': 'from langchain_google_community import VertexFSVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_google_community.VertexFSVectorStore']}, 'Hippo': {'imports': 'from langchain_community.vectorstores.hippo import Hippo', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.hippo.Hippo']}, 'Hologres': {'imports': 'from langchain_community.vectorstores import Hologres', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Hologres']}, 'Infinispan': {'imports': 'from langchain_community.vectorstores import InfinispanVS', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.InfinispanVS']}, 'Jaguar Vector Database': {'imports': 'from langchain_community.vectorstores.jaguar import Jaguar', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.jaguar.Jaguar']}, 'KDB.AI': {'imports': 'from langchain_community.vectorstores import KDBAI', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.KDBAI']}, 'Kinetica Vectorstore API': {'imports': 'from langchain_community.vectorstores import Kinetica', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Kinetica']}, 'LanceDB': {'imports': 'from langchain_community.vectorstores import LanceDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.LanceDB']}, 'Lantern': {'imports': 'from langchain_community.vectorstores import Lantern', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Lantern']}, 'LindormVectorStore': {'imports': 'from langchain_lindorm_integration.vectorstores import LindormVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_lindorm_integration.vectorstores.LindormVectorStore']}, 'LLMRails': {'imports': 'from langchain_community.vectorstores import LLMRails', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.LLMRails']}, 'ManticoreSearch VectorStore': {'imports': 'from langchain_community.vectorstores import ManticoreSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.ManticoreSearch']}, 'Marqo': {'imports': 'from langchain_community.vectorstores import Marqo', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Marqo']}, 'Meilisearch': {'imports': 'from langchain_community.vectorstores import Meilisearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Meilisearch']}, 'Amazon MemoryDB': {'imports': 'from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_aws.vectorstores.inmemorydb.InMemoryVectorStore']}, 'Milvus': {'imports': 'from langchain_milvus import Milvus', 'category': 'VECTOR_STORE', 'import_list': ['langchain_milvus.Milvus']}, 'Momento Vector Index': {'imports': 'from langchain_community.vectorstores import MomentoVectorIndex', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.MomentoVectorIndex']}, 'MongoDB Atlas': {'imports': 'from langchain_mongodb import MongoDBAtlasVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_mongodb.MongoDBAtlasVectorSearch']}, 'MyScale': {'imports': 'from langchain_community.vectorstores import MyScale', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.MyScale']}, 'Neo4j Vector Index': {'imports': 'from langchain_neo4j import Neo4jVector', 'category': 'VECTOR_STORE', 'import_list': ['langchain_neo4j.Neo4jVector']}, 'NucliaDB': {'imports': 'from langchain_community.vectorstores.nucliadb import NucliaDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.nucliadb.NucliaDB']}, 'OceanbaseVectorStore': {'imports': 'from langchain_oceanbase.vectorstores import OceanbaseVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_oceanbase.vectorstores.OceanbaseVectorStore']}, 'OpenSearch': {'imports': 'from langchain_community.vectorstores import OpenSearchVectorSearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.OpenSearchVectorSearch']}, 'Oracle AI Vector Search: Vector Store': {'imports': 'from langchain_community.vectorstores.oraclevs import OracleVS', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.oraclevs.OracleVS']}, 'Pathway': {'imports': 'from langchain_community.vectorstores import PathwayVectorClient', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.PathwayVectorClient']}, 'Postgres Embedding': {'imports': 'from langchain_community.vectorstores import PGEmbedding', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.PGEmbedding']}, 'PGVecto.rs': {'imports': 'from langchain_community.vectorstores.pgvecto_rs import PGVecto_rs', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.pgvecto_rs.PGVecto_rs']}, 'PGVector': {'imports': 'from langchain_postgres.vectorstores import PGVector', 'category': 'VECTOR_STORE', 'import_list': ['langchain_postgres.vectorstores.PGVector']}, 'Pinecone': {'imports': 'from langchain_pinecone import PineconeVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_pinecone.PineconeVectorStore']}, 'Qdrant': {'imports': 'from langchain_qdrant import QdrantVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_qdrant.QdrantVectorStore']}, 'Redis Vector Store': {'imports': 'from langchain_redis import RedisVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_redis.RedisVectorStore']}, 'Relyt': {'imports': 'from langchain_community.vectorstores import Relyt', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Relyt']}, 'SAP HANA Cloud Vector Engine': {'imports': 'from langchain_community.vectorstores.hanavector import HanaDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.hanavector.HanaDB']}, 'ScaNN': {'imports': 'from langchain_community.vectorstores import ScaNN', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.ScaNN']}, 'SemaDB': {'imports': 'from langchain_community.vectorstores import SemaDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SemaDB']}, 'scikit-learn': {'imports': 'from langchain_community.vectorstores import SKLearnVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SKLearnVectorStore']}, 'SQLiteVec': {'imports': 'from langchain_community.vectorstores import SQLiteVec', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SQLiteVec']}, 'SQLite-VSS': {'imports': 'from langchain_community.vectorstores import SQLiteVSS', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SQLiteVSS']}, 'SQLServer': {'imports': 'from langchain_sqlserver import SQLServer_VectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_sqlserver.SQLServer_VectorStore']}, 'StarRocks': {'imports': 'from langchain_community.vectorstores import StarRocks', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.StarRocks']}, 'Supabase (Postgres)': {'imports': 'from langchain_community.vectorstores import SupabaseVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.SupabaseVectorStore']}, 'Tablestore': {'imports': 'from langchain_community.vectorstores import TablestoreVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.TablestoreVectorStore']}, 'Tair': {'imports': 'from langchain_community.vectorstores import Tair', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Tair']}, 'Tencent Cloud VectorDB': {'imports': 'from langchain_community.vectorstores import TencentVectorDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.TencentVectorDB']}, 'ThirdAI NeuralDB': {'imports': 'from langchain_community.vectorstores import NeuralDBVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.NeuralDBVectorStore']}, 'TiDB Vector': {'imports': 'from langchain_community.vectorstores import TiDBVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.TiDBVectorStore']}, 'Tigris': {'imports': 'from langchain_community.vectorstores import Tigris', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Tigris']}, 'TileDB': {'imports': 'from langchain_community.vectorstores import TileDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.TileDB']}, 'Timescale Vector (Postgres)': {'imports': 'from langchain_community.vectorstores.timescalevector import TimescaleVector', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.timescalevector.TimescaleVector']}, 'Typesense': {'imports': 'from langchain_community.vectorstores import Typesense', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Typesense']}, 'Upstash Vector': {'imports': 'from langchain_community.vectorstores.upstash import UpstashVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.upstash.UpstashVectorStore']}, 'USearch': {'imports': 'from langchain_community.vectorstores import USearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.USearch']}, 'Vald': {'imports': 'from langchain_community.vectorstores import Vald', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Vald']}, "Intel's Visual Data Management System (VDMS)": {'imports': 'from langchain_community.vectorstores import VDMS\nor\nfrom langchain_community.vectorstores.vdms import VDMS_Client', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.VDMS', 'langchain_community.vectorstores.vdms.VDMS_Client']}, 'Vearch': {'imports': 'from langchain_community.vectorstores.vearch import Vearch', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.vearch.Vearch']}, 'Vectara': {'imports': 'from langchain_community.vectorstores import Vectara', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Vectara']}, 'viking DB': {'imports': 'from langchain_community.vectorstores.vikingdb import VikingDB', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.vikingdb.VikingDB']}, 'vlite': {'imports': 'from langchain_community.vectorstores import VLite', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.VLite']}, 'Weaviate': {'imports': 'from langchain_weaviate.vectorstores import WeaviateVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_weaviate.vectorstores.WeaviateVectorStore']}, 'Xata': {'imports': 'from langchain_community.vectorstores.xata import XataVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.xata.XataVectorStore']}, 'Yellowbrick': {'imports': 'from langchain_community.vectorstores import Yellowbrick', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Yellowbrick']}, 'Zep': {'imports': 'from langchain_community.vectorstores import ZepVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.ZepVectorStore']}, 'Zilliz': {'imports': 'from langchain_community.vectorstores import Milvus', 'category': 'VECTOR_STORE', 'import_list': ['langchain_community.vectorstores.Milvus']}}

In [463]:
found_imports = parse_all_imports_from_directory(root_directory)

possible_tools = []

for name, values in predefined_tools_dict.items():

    if any(single_import in found_imports for single_import in values["import_list"]):
        new_possible_tool = {"name": name}
        for value_name, value_content in values.items():
            new_possible_tool[value_name] = value_content
        possible_tools.append(new_possible_tool)
    
custom_tools = parse_all_custom_tools_from_directory(root_directory)

In [464]:
print(possible_tools)

[{'name': 'Jira Toolkit', 'imports': 'from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit\nor\nfrom langchain_community.utilities.jira import JiraAPIWrapper\n', 'category': 'GENERAL', 'import_list': ['langchain_community.agent_toolkits.jira.toolkit.JiraToolkit', 'langchain_community.utilities.jira.JiraAPIWrapper']}, {'name': 'Tavily Search', 'imports': 'from langchain_community.tools import TavilySearchResults', 'category': 'GENERAL', 'import_list': ['langchain_community.tools.TavilySearchResults']}, {'name': 'Azure OpenAI', 'imports': 'from langchain_openai import AzureChatOpenAI', 'category': 'LLM', 'import_list': ['langchain_openai.AzureChatOpenAI']}, {'name': 'AWS Bedrock', 'imports': 'from langchain_aws import ChatBedrock', 'category': 'LLM', 'import_list': ['langchain_aws.ChatBedrock']}, {'name': 'Pinecone', 'imports': 'from langchain_pinecone import PineconeVectorStore', 'category': 'VECTOR_STORE', 'import_list': ['langchain_pinecone.PineconeVectorStore']}]


In [465]:
print(custom_tools)

[{'name': 'generate_document_parsed_input_json_tool', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'retriever_tool', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'run_query', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'search_issue', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'create_issue', 'filepath': './Gladiator2\\Tools\\create_tools.py'}]


In [466]:
import ast
import os

def find_package_root(file_path):
    """
    Given a file path, search upward for the highest directory that contains
    an __init__.py file. If no __init__.py is found in the file's directory,
    returns None.
    """
    directory = os.path.dirname(file_path)
    package_root = None
    while True:
        if os.path.exists(os.path.join(directory, '__init__.py')):
            package_root = directory  # update to current package dir
            parent_directory = os.path.dirname(directory)
            if parent_directory == directory:
                break  # reached filesystem root
            directory = parent_directory
        else:
            break
    return package_root

def module_name_from_filepath(file_path, package_root):
    """
    Convert a file path into a module name relative to the package root.
    For example, if package_root is /path/to/my_package and
    file_path is /path/to/my_package/sub/module.py, this returns "sub.module".
    
    If no package_root is found (i.e. file isn't in a package), fall back to
    using the filename (without extension) as the module name.
    """
    if package_root:
        rel_path = os.path.relpath(file_path, package_root)
        mod_name, _ = os.path.splitext(rel_path)
        return mod_name.replace(os.path.sep, '.')
    else:
        return os.path.splitext(os.path.basename(file_path))[0]

class ToolVisitor(ast.NodeVisitor):
    def __init__(self, module_name, file_path):
        self.module_name = module_name
        self.file_path = file_path
        self.scope_stack = []  # To keep track of enclosing class names.
        self.tools = []

    def visit_ClassDef(self, node):
        # Entering a class: push its name onto the scope stack.
        self.scope_stack.append(node.name)
        self.generic_visit(node)
        self.scope_stack.pop()

    def visit_FunctionDef(self, node):
        decorator_names = []
        for decorator in node.decorator_list:
            if isinstance(decorator, ast.Name):
                decorator_names.append(decorator.id)
            elif isinstance(decorator, ast.Call):
                if isinstance(decorator.func, ast.Name):
                    decorator_names.append(decorator.func.id)
                elif isinstance(decorator.func, ast.Attribute):
                    decorator_names.append(decorator.func.attr)

        if "tool" in decorator_names:
            # Build a fully qualified name: module + any class scopes + function name.
            full_name_parts = [self.module_name] if self.module_name else []
            full_name_parts.extend(self.scope_stack)
            full_name_parts.append(node.name)
            fully_qualified_name = ".".join(full_name_parts)
            self.tools.append({
                "name": node.name,
                "fully_qualified_name": fully_qualified_name,
                "filepath": self.file_path
            })

        self.generic_visit(node)

def extract_custom_tools_with_ast(file_content, file_path):
    """
    Parse the file content, automatically determine the package root,
    and extract all functions decorated with @tool along with their
    fully qualified names.
    """
    tree = ast.parse(file_content)
    package_root = find_package_root(file_path)
    module_name = module_name_from_filepath(file_path, package_root)
    visitor = ToolVisitor(module_name, file_path)
    visitor.visit(tree)
    return visitor.tools

def parse_all_custom_tools_from_directory(directory_path):
    all_custom_tools = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    file_content = f.read()
                    custom_tools = extract_custom_tools_with_ast(file_content, file_path)
                    all_custom_tools.extend(custom_tools)
    return all_custom_tools

In [467]:
custom_tools = parse_all_custom_tools_from_directory(root_directory)
print(custom_tools)

[{'name': 'generate_document_parsed_input_json_tool', 'fully_qualified_name': 'Tools.create_tools.generate_document_parsed_input_json_tool', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'retriever_tool', 'fully_qualified_name': 'Tools.create_tools.retriever_tool', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'run_query', 'fully_qualified_name': 'Tools.create_tools.run_query', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'search_issue', 'fully_qualified_name': 'Tools.create_tools.search_issue', 'filepath': './Gladiator2\\Tools\\create_tools.py'}, {'name': 'create_issue', 'fully_qualified_name': 'Tools.create_tools.create_issue', 'filepath': './Gladiator2\\Tools\\create_tools.py'}]


In [468]:
def parse_all_graph_instances_in_directory(root_directory, graph_class_fqcn, add_conditional_edges_method_name, add_node_method_name):

    results = walk_directory_and_parse(root_directory, graph_class_fqcn, add_conditional_edges_method_name, add_node_method_name)
    graphs = []
    for graph, call_records in results.items():
        nodes = []
        basic_edges = []
        conditional_edges = []

        if call_records.get("add_node", False):
            call_data = call_records.get("add_node")
            for single_call in call_data:
                # The actual name if the method received 2 arguments, and the name of the function if it received one
                node_name = single_call["positional"][0]
                node_definition = single_call["node_definition_argument_info"]
                nodes.append({
                    "name": node_name,
                    "definition": node_definition
                })
            
        if call_records.get("add_edge", False):
            all_node_names = [node["name"] for node in nodes] + ["START", "END"]
            print(all_node_names)
            call_data = call_records.get("add_edge")
            for single_call in call_data:
                nodes_in_edge = []
                # Since the call always contains two node names, we just add names from positional and keyword arguments (in this exact order)
                for node_name in single_call["positional"]:
                    nodes_in_edge.append(node_name)
                for _, node_name in single_call["keyword"].items():
                    nodes_in_edge.append(node_name)

                if nodes_in_edge[0] in all_node_names and nodes_in_edge[1] in all_node_names:
                    basic_edges.append({
                        "start_node": nodes_in_edge[0],
                        "end_node": nodes_in_edge[1]
                    })

        if call_records.get("add_conditional_edges", False):
            all_node_names = [node["name"] for node in nodes] + ["START", "END"]
            call_data = call_records.get("add_conditional_edges")
            for single_call in call_data:
                # Resolving arguments to find the source node
                arguments = []
                for argument in single_call["positional"]:
                    arguments.append(argument)
                for _, argument in single_call["keyword"].items():
                    arguments.append(argument)
                
                if single_call.get("path", False):
                    if single_call["path"].get("function_returns", False):
                        for end_node in single_call["path"].get("function_returns"):
                            if arguments[0] in all_node_names and end_node in all_node_names:
                                conditional_edges.append({
                                    "resolved": True,
                                    "start_node": arguments[0],
                                    "end_node": end_node
                                })
                    elif single_call["path"].get("function_fq_name", False):
                        conditional_edges.append({
                            "resolved": False,
                            "function_fq_name": single_call["path"].get("function_fq_name")
                        })
                elif single_call.get("path_map", False):
                    if single_call["path_map"].get("list_values", False):
                        for end_node in single_call["path_map"].get("list_values"):
                            if arguments[0] in all_node_names and end_node in all_node_names:
                                conditional_edges.append({
                                    "resolved": True,
                                    "start_node": arguments[0],
                                    "end_node": end_node
                                })
                    elif single_call["path_map"].get("dict_values", False):
                        for end_node in single_call["path_map"].get("dict_values"):
                            if arguments[0] in all_node_names and end_node in all_node_names:
                                conditional_edges.append({
                                    "resolved": True,
                                    "start_node": arguments[0],
                                    "end_node": end_node
                                })
                    elif single_call["path_map"].get("map_fq_name", False):
                        conditional_edges.append({
                            "resolved": False,
                            "map_fq_name": single_call["path_map"].get("map_fq_name")
                        })
        
        graphs.append({
            "graph_name": graph,
            "graph_file_path": call_records["filepath"],
            "graph_info": {
                "nodes": nodes,
                "basic_edges": basic_edges,
                "conditional_edges": conditional_edges
            }
        })
    return graphs


In [469]:
results = parse_all_graph_instances_in_directory(root_directory, graph_class_fqcn, add_conditional_edges_method_name, add_node_method_name)

In [470]:
print(results)

[{'graph_name': 'workflow', 'graph_file_path': './Gladiator2\\graph.py', 'graph_info': {'nodes': [{'name': 'input_node', 'definition': {'original': 'input_node', 'fq_name': 'nodes.nodes.input_node'}}, {'name': 'researcher', 'definition': {'original': 'call_model', 'fq_name': 'nodes.researcher.call_model'}}, {'name': 'tools', 'definition': {'original': 'tool_node', 'fq_name': 'nodes.nodes.tool_node'}}, {'name': 'generator', 'definition': {'original': 'generate_document_node', 'fq_name': 'nodes.generator.generate_document_node'}}, {'name': 'output_node', 'definition': {'original': 'output_node', 'fq_name': 'nodes.nodes.output_node'}}], 'basic_edges': [], 'conditional_edges': []}}]


In [471]:
def get_python_files(directory):
    python_files = []
    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a .py extension
            if file.endswith('.py'):
                # Append the full path to the list
                python_files.append(os.path.join(root, file))
    return python_files

print(get_python_files(root_directory))

['./Gladiator2\\api.py', './Gladiator2\\graph.py', './Gladiator2\\legal_chatbot.py', './Gladiator2\\__init__.py', './Gladiator2\\example\\dicts.py', './Gladiator2\\example\\func.py', './Gladiator2\\LLMs\\llm.py', './Gladiator2\\LLMs\\__init__.py', './Gladiator2\\nodes\\generator.py', './Gladiator2\\nodes\\nodes.py', './Gladiator2\\nodes\\researcher.py', './Gladiator2\\nodes\\state.py', './Gladiator2\\nodes\\__init__.py', './Gladiator2\\prompts\\prompts.py', './Gladiator2\\prompts\\__init__.py', './Gladiator2\\Tools\\create_tools.py', './Gladiator2\\Tools\\helper.py', './Gladiator2\\Tools\\tools.py', './Gladiator2\\Tools\\__init__.py']


In [472]:
def parse_gotos(node, graph_file_path, root_directory):
    
    with open(graph_file_path, "r") as f:
        code = f.read()

    system_prompt = """# Role Definition
You are an analyzer of Python functions.
Your goal is to detect if a Python function returns a specific type of object and then provide a list of all the possible values of a certain argument of that object.
You are currently looking for returning the Command object, and you need to provide a list of all the possible values of the "goto" argument of that object.

# Instructions
In the first user message you will be provided with the following information:
- the code file in which the function is used as an argument to a "add_node" method between the # Code Beginning and # Code End separators
- the path to that file between the # Code File Path Beginning and # Code File Path End separators
- the name of the function as it is passed to the "add_node" method, or the AST dump of the function as it is passed to the "add_node" method between the # Function Original Name Beginning and # Function Original Name End separators
- the fully qualified name of the function between the # Fully Qualified Name Beginning and # Fully Qualified Name End separators
- the paths to all of the available files in the directory between the # File Paths Beginning and # File Paths End separators

# Task
Your job is to look for the definition of the function in the code files and see if the function returns any Command objects.
If the function returns any Command objects you must provide a list of all the possible values of the Command object's "goto" argument. The values of the "goto" argument can only be strings, END or START.
Some functions do not return any Command objects, and in those cases you should provide an empty list.
Only focus on the single function you were assigned in the first user message.

# Response format
You must respond with a JSON containing the following fields:
1. A "need_more_information" field". This is a boolean and it should be true if you need to see the code from some of the other files in the directory in order to solve the task, and false otherwise.
2. A "file_paths" filed. This is a list of strings of the file paths of files from which you need to see the code. These can only be file paths from between the # File Paths Beginning and # File Paths End separators in the first user message. It should be an empty list if the "need_more_information" field is set to false.
3. A "goto_values" list. This should contain all the possible values of the "goto" argument of the Command objects. If the function does not return any Command objects, this should be an empty list."""

    client = openai.AzureOpenAI()

    first_user_message = f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{graph_file_path}
# Code File Path End

# Function Original Name Beginning
{node["definition"]["original"]}
# Function Original Name End

# Fully Qualified Name Beginning
{node["definition"]["fq_name"]}
# Fully Qualified Name End

# File Paths Beginning
{get_python_files(root_directory)}
# File Paths End

"""

    messages= [
        {"role":"system", "content": system_prompt},
        {"role": "user", "content": first_user_message}
    ]

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages= messages,
        response_format={"type":"json_object"}
    )

    response_json = json.loads(response.choices[0].message.content)

    print(response_json)

    need_more_information = response_json["need_more_information"]

    while need_more_information:
        new_user_message = ""
        for file_path in response_json["file_paths"]:

            with open(file_path, "r") as f:
                code = f.read()
                new_user_message+=f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{file_path}
# Code File Path End

"""
        
        messages.append({
            "role":"assistant",
            "content": response.choices[0].message.content
        })
        messages.append({
            "role": "user",
            "content": new_user_message
        })

        response = client.chat.completions.create(
            model = "gpt-4o",
            messages= messages,
            response_format={"type":"json_object"}
        )

        response_json = json.loads(response.choices[0].message.content)

        print(response_json)

        need_more_information = response_json["need_more_information"]


In [473]:
# for graph in results:

#     print(f"Processing nodes from {graph["graph_name"]}")

#     for node in graph["graph_info"]["nodes"]:
#         print(f"Node: {node["name"]}")
#         parse_gotos(node, graph["graph_file_path"], root_directory)


In [474]:
def parse_tool_access(node, graph_file_path, root_directory, predefined_tools, custom_tools):
    
    with open(graph_file_path, "r") as f:
        code = f.read()

    system_prompt = """# Role Definition
You are an analyzer of Python functions.
Your goal is to fully trace the execution of a Python function, following every function call, class instantiation, method invocation, and variable definition, even when they lead to other source code files. Furthermore, if any of these definitions or calls contain other definitions or calls, you must track them recursively. You are finished with this once you have exhausted and followed all the definitions and calls, ensuring a comprehensive understanding of all dependencies and interactions within the codebase.
You are doing this in order to understand which predefined tools and custom tools the function uses.
Predefined tools are tools that are already defined in external libraries or modules, and are used by importing them.
Custom tools are functions that are decorated with the @tool decorator, and that are defined within this codebase.

# Instructions
In the first user message you will be provided with the following information:
- the code file in which the function is used as an argument to a "add_node" method between the # Code Beginning and # Code End separators
- the path to that file between the # Code File Path Beginning and # Code File Path End separators
- the name of the function as it is passed to the "add_node" method, or the AST dump of the function as it is passed to the "add_node" method between the # Function Original Name Beginning and # Function Original Name End separators
- the fully qualified name of the function between the # Fully Qualified Name Beginning and # Fully Qualified Name End separators
- the paths to all of the available files in the directory between the # File Paths Beginning and # File Paths End separators
- a list of all the predefined tools whose usage has been detected in the directory between the # Predefined Tools List Beginning and # Predefined Tools List End separators
- a list of all the custom tools that are defined in the directory between the # Custom Tools List Beginning and # Custom Tools List End separators

# Task
Your job is to look for the definition of the function in the code files and then fully trace it's execution in order to understand which predefined tools and custom tools the function uses.
Predefined Tools will be listed between the # Predefined Tools List Beginning and # Predefined Tools List End separators. Each predefined tool in the list will be defined by the tool's name, the category to which the tool belongs, and the imports needed to use the predefined tool. The imports should help you detect where a predefined tool is used.
Custom Tools will be listed between the # Custom Tools List Beginning and # Custom Tools List End separators. Each custom tool in the list will be defined by the tool's name (which is the name of the function decorated with the @tool decorator), the category to which the tool belongs (which is always "CUSTOM" for custom tools), the fully qualified name of the custom tool, and the file path of the file in which the custom tool is defined.
Pay attention to tools from the "LLM" category of tools because you need to detect if these tools ever have the "bind_tools" method called on them.
Some functions do not use any predefined or custom tools.
Only focus on the single function you were assigned in the first user message.

# Possible Scenarios
1. The first possible scenario is that a function reference is passed to the "add_node" method. In this case you must simply find the defintion of the function that has been passed and trace its execution.
2. The second possible scenario is that a function call is passed to the "add_node" method. In this case you must focus on the function or class that the function call returns, and trace its execution.
3. If a ToolNode object is passed to the "add_node" method, or returned by the function call passed to the "add_node" method, then you must find the list of tools that the ToolNode uses.
4. If any other class is passed to the "add_node" method, or is returned by the function call passed to the "add_node" method, then you must trace the execution of the __call__ method of that class.

# Response format
You must respond with a JSON containing the following fields:
1. A "notes" field. This is a short note to yourself in the future in which you explain which function, class, method or variable definitions you still have to look at in order to solve the task, and explain your thoughts about the task so far. Keep this precise, as you will rely on this in the future.
2. A "need_more_information" field. This is a boolean and it should be true if you need to see the code from some of the other files in the directory in order to solve the task, and false otherwise. The code from the requested files will be provided in the next user message.
3. A "file_paths" filed. This is a list of strings of the file paths of files from which you need to see the code. These can only be file paths from between the # File Paths Beginning and # File Paths End separators in the first user message. It should be an empty list if the "need_more_information" field is set to false.
4. A "predefined_tools" list. This should contain JSONs of all the predefined tools from the list of predefined tools in the first user message, and information about whether or not the function uses them. Each tool JSON should contain six fields. The first field is "ordinal_id" which is the same as the predefined tool's ordinal id in the list of predefined tools. You can use this to keep track of whether you listed all of the tools from the predefined tools list. The second field is "name", which should contain the name of the tool from the list of predefined tools. The third field is "category", which should contain the category of the tool from the list of predefined tools. The fourth field is "bind_tools", which should be true if the tool is from the "LLM" category and has the "bind_tools" method called, and false in all other cases. The fifth field is "usage_explanation" in which you explain the execution trace through which the function uses this predefined tool, or not if it doesn't. The sixth field is "used", which is a boolean that should be set to True if the function uses this predefined tool, and False if it does not.
5. A "custom_tools" list. This should contain JSONs of all the custom tools from the list of custom tools in the first user message, and information about whether or not the function uses them. Each tool JSON should contain four fields. The first field is "ordinal_id" which is the same as the custom tool's ordinal id in the list of custom tools. You can use this to keep track of whether you listed all of the tools from the custom tools list. The second field is "name", which should contain the name of the tool from the list of custom tools. The third field is "usage_explanation" in which you explain the execution trace through which the function uses this custom tool, or not if it doesn't. The fourth field is "used", which is a boolean that should be set to True if the function uses this custom tool, and False if it does not.
"""

    client = openai.AzureOpenAI()

    first_user_message = f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{graph_file_path}
# Code File Path End

# Function Original Name Beginning
{node["definition"]["original"]}
# Function Original Name End

# Fully Qualified Name Beginning
{node["definition"]["fq_name"]}
# Fully Qualified Name End

# File Paths Beginning
{get_python_files(root_directory)}
# File Paths End

"""
    
    first_user_message+=f"""# Predefined Tools List Beginning
"""
    for i, predefined_tool in enumerate(predefined_tools):

        first_user_message+=f"""{i+1}. Tool Name: {predefined_tool["name"]}
Tool Category: {predefined_tool["category"]}
Tool Imports: {predefined_tool["imports"]}

"""

    first_user_message+=f"""# Predefined Tools List End
"""
    
    first_user_message+=f"""# Custom Tools List Beginning
"""
    for i, custom_tool in enumerate(custom_tools):

        first_user_message+=f"""{i+1}. Tool Name: {custom_tool["name"]}
Tool Category: CUSTOM
Tool Fully Qualified Name: {custom_tool["fully_qualified_name"]}
Tool Fully File Path: {custom_tool["filepath"]}

"""

    first_user_message+=f"""# Custom Tools List End
"""

    messages= [
        {"role":"system", "content": system_prompt},
        {"role": "user", "content": first_user_message}
    ]

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages= messages,
        response_format={"type":"json_object"}
    )

    response_json = json.loads(response.choices[0].message.content)

    print(response_json)

    need_more_information = response_json["need_more_information"]

    while need_more_information:
        new_user_message = ""
        for file_path in response_json["file_paths"]:

            with open(file_path, "r") as f:
                code = f.read()
                new_user_message+=f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{file_path}
# Code File Path End

"""
        
        messages.append({
            "role":"assistant",
            "content": response.choices[0].message.content
        })
        messages.append({
            "role": "user",
            "content": new_user_message
        })

        response = client.chat.completions.create(
            model = "gpt-4o",
            messages= messages,
            response_format={"type":"json_object"}
        )

        response_json = json.loads(response.choices[0].message.content)

        print(response_json)

        need_more_information = response_json["need_more_information"]
    
    for predefined_tool in response_json["predefined_tools"]:
        if predefined_tool["used"]:
            print(predefined_tool["name"])

    for custom_tool in response_json["custom_tools"]:
        if custom_tool["used"]:
            print(custom_tool["name"])
    

In [475]:
for graph in results:

    print(f"Processing nodes from {graph["graph_name"]}")

    for node in graph["graph_info"]["nodes"]:
        print(f"Node: {node["name"]}")
        parse_tool_access(node, graph["graph_file_path"], root_directory, possible_tools, custom_tools)

Processing nodes from workflow
Node: input_node


KeyboardInterrupt: 

In [ ]:
def parse_tool_access(node, graph_file_path, root_directory, predefined_tools, custom_tools):
    
    with open(graph_file_path, "r") as f:
        code = f.read()

    system_prompt = """# Role Definition
You are an analyzer of Python functions.
Your goal is split into two parts:
1. The first part is to fully trace the execution of a function. This means that you should look at the functions definition, identify all the functions, classes, methods, and variables that the function uses, and find the definitions of these functions, classes, methods, and variables by asking to see the content of the code files in which they are defined. Furthermore, if these components contain any other functions, classes, methods, and variables, you should continue this process recursively until you have access to all the definitions.
2. The second part is seeing which predefined tools and custom tools, if any, the function uses during its execution. You will be provided with the information needed to detect these tools.
Predefined tools are tools that are already defined in external libraries or modules, and are used by importing them.
Custom tools are functions that are decorated with the @tool decorator, and that are defined within this codebase.

# Instructions
In the first user message you will be provided with the following information:
- the code file in which the function is used as an argument to a "add_node" method between the # Code Beginning and # Code End separators
- the path to that file between the # Code File Path Beginning and # Code File Path End separators
- the name of the function as it is passed to the "add_node" method, or the AST dump of the function as it is passed to the "add_node" method between the # Function Original Name Beginning and # Function Original Name End separators
- the fully qualified name of the function between the # Fully Qualified Name Beginning and # Fully Qualified Name End separators
- the paths to all of the available files in the directory between the # File Paths Beginning and # File Paths End separators
- a list of all the predefined tools whose usage has been detected in the directory between the # Predefined Tools List Beginning and # Predefined Tools List End separators
- a list of all the custom tools that are defined in the directory between the # Custom Tools List Beginning and # Custom Tools List End separators

# Task
Your job is to look for the definition of the function in the code files and then fully trace its execution in order to understand which predefined tools and custom tools the function uses.
Predefined Tools will be listed between the # Predefined Tools List Beginning and # Predefined Tools List End separators. Each predefined tool in the list will be defined by the tool's name, the category to which the tool belongs, and the imports needed to use the predefined tool. The imports should help you detect where a predefined tool is used.
Custom Tools will be listed between the # Custom Tools List Beginning and # Custom Tools List End separators. Each custom tool in the list will be defined by the tool's name (which is the name of the function decorated with the @tool decorator), the category to which the tool belongs (which is always "CUSTOM" for custom tools), the fully qualified name of the custom tool, and the file path of the file in which the custom tool is defined.
Pay attention to tools from the "LLM" category of predefined tools because you need to detect if these tools ever have the "bind_tools" method called on them.
Some functions do not use any predefined or custom tools.
Only focus on the execution of the single function you were assigned in the first user message.

# Possible Scenarios
These are the scenarios you might encounter while looking at the "add_node" method:
1. The first possible scenario is that a function reference is passed to the "add_node" method. In this case you must simply find the defintion of the function that has been passed and trace its execution.
2. The second possible scenario is that a function call is passed to the "add_node" method. In this case you must focus on the function or class that the function call returns, and trace its execution.
3. If a ToolNode object is passed to the "add_node" method, or returned by the function call passed to the "add_node" method, then you must find the list of tools that the ToolNode uses.
4. If any other class is passed to the "add_node" method, or is returned by the function call passed to the "add_node" method, then you must trace the execution of the __call__ method of that class.

# Response format
You must respond with a JSON containing the following fields:
1. A "notes" field. This is a short note to yourself in the future in which you explain which function, class, method or variable definitions you still have to look at in order to solve the task, and explain your thoughts about the task so far. Keep this precise, as you will rely on this in the future.
2. A "need_more_information" field. This is a boolean and it should be true if you need to see the code from some of the other files in the directory in order to solve the task, and false otherwise.
3. A "file_paths" filed. This is a list of strings of the file paths of files from which you need to see the code. These can only be file paths from between the # File Paths Beginning and # File Paths End separators in the first user message. It should be an empty list if the "need_more_information" field is set to false. The code from the requested files will be provided in the next user message.
4. A "predefined_tools" list. This should contain JSONs of all the predefined tools from the list of predefined tools in the first user message, and information about whether or not the function uses them. Each tool JSON should contain six fields. The first field is "ordinal_id" which is the same as the predefined tool's ordinal id in the list of predefined tools. You can use this to keep track of whether you listed all of the tools from the predefined tools list. The second field is "name", which should contain the name of the tool from the list of predefined tools. The third field is "category", which should contain the category of the tool from the list of predefined tools. The fourth field is "bind_tools", which should be true if the tool is from the "LLM" category and has the "bind_tools" method called, and false in all other cases. The fifth field is "usage_explanation" in which you explain the execution trace through which the function uses this predefined tool, or not if it doesn't. The sixth field is "used", which is a boolean that should be set to True if the function uses this predefined tool, and False if it does not. This should be an empty list if the "need_more_information" field is set to True.
5. A "custom_tools" list. This should contain JSONs of all the custom tools from the list of custom tools in the first user message, and information about whether or not the function uses them. Each tool JSON should contain four fields. The first field is "ordinal_id" which is the same as the custom tool's ordinal id in the list of custom tools. You can use this to keep track of whether you listed all of the tools from the custom tools list. The second field is "name", which should contain the name of the tool from the list of custom tools. The third field is "usage_explanation" in which you explain the execution trace through which the function uses this custom tool, or not if it doesn't. The fourth field is "used", which is a boolean that should be set to True if the function uses this custom tool, and False if it does not. This should be an empty list if the "need_more_information" field is set to True.
"""

    client = openai.AzureOpenAI()

    first_user_message = f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{graph_file_path}
# Code File Path End

# Function Original Name Beginning
{node["definition"]["original"]}
# Function Original Name End

# Fully Qualified Name Beginning
{node["definition"]["fq_name"]}
# Fully Qualified Name End

# File Paths Beginning
{get_python_files(root_directory)}
# File Paths End

"""
    
    first_user_message+=f"""# Predefined Tools List Beginning
"""
    for i, predefined_tool in enumerate(predefined_tools):

        first_user_message+=f"""{i+1}. Tool Name: {predefined_tool["name"]}
Tool Category: {predefined_tool["category"]}
Tool Imports: {predefined_tool["imports"]}

"""

    first_user_message+=f"""# Predefined Tools List End
"""
    
    first_user_message+=f"""# Custom Tools List Beginning
"""
    for i, custom_tool in enumerate(custom_tools):

        first_user_message+=f"""{i+1}. Tool Name: {custom_tool["name"]}
Tool Category: CUSTOM
Tool Fully Qualified Name: {custom_tool["fully_qualified_name"]}
Tool Fully File Path: {custom_tool["filepath"]}

"""

    first_user_message+=f"""# Custom Tools List End
"""

    messages= [
        {"role":"system", "content": system_prompt},
        {"role": "user", "content": first_user_message}
    ]

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages= messages,
        response_format={"type":"json_object"}
    )

    response_json = json.loads(response.choices[0].message.content)

    print(response_json)

    need_more_information = response_json["need_more_information"]

    while need_more_information:
        new_user_message = ""
        for file_path in response_json["file_paths"]:

            with open(file_path, "r") as f:
                code = f.read()
                new_user_message+=f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{file_path}
# Code File Path End

"""
        
        messages.append({
            "role":"assistant",
            "content": response.choices[0].message.content
        })
        messages.append({
            "role": "user",
            "content": new_user_message
        })

        response = client.chat.completions.create(
            model = "gpt-4o",
            messages= messages,
            response_format={"type":"json_object"}
        )

        response_json = json.loads(response.choices[0].message.content)

        print(response_json)

        need_more_information = response_json["need_more_information"]
    
    for predefined_tool in response_json["predefined_tools"]:
        if predefined_tool["used"]:
            print(predefined_tool["name"])

    for custom_tool in response_json["custom_tools"]:
        if custom_tool["used"]:
            print(custom_tool["name"])

In [ ]:
for graph in results:

    print(f"Processing nodes from {graph["graph_name"]}")

    for node in graph["graph_info"]["nodes"]:
        print(f"Node: {node["name"]}")
        parse_tool_access(node, graph["graph_file_path"], root_directory, possible_tools, custom_tools)

Processing nodes from workflow
Node: input_node
{'notes': "I need to examine the definition of the 'input_node' function in the 'nodes/nodes.py' file to trace its execution and determine which predefined and custom tools it uses.", 'need_more_information': True, 'file_paths': ['./Gladiator2/nodes/nodes.py'], 'predefined_tools': [], 'custom_tools': []}
{'notes': "I have found the definition of the 'input_node' function. To fully trace its execution, I need to check the 'add_documents' method of the 'legal_docs_vector_store_class' in the 'Tools/tools.py' file because it is used by the 'input_node' function.", 'need_more_information': True, 'file_paths': ['./Gladiator2/Tools/tools.py'], 'predefined_tools': [], 'custom_tools': []}
{'notes': "I have identified the definition of the 'legal_docs_vector_store_class' and the 'HuggingFaceEmbeddings' tool as well as the 'TavilySearchResults' predefined tools' usage.  Since the 'legal_docs_vector_store_class' uses 'LegalDocumentsVectorStore' from 

In [ ]:
def parse_tool_access(node, graph_file_path, root_directory, predefined_tools, custom_tools):
    
    with open(graph_file_path, "r") as f:
        code = f.read()

    system_prompt = """"""

    client = openai.AzureOpenAI()

    first_user_message = f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{graph_file_path}
# Code File Path End

# Function Original Name Beginning
{node["definition"]["original"]}
# Function Original Name End

# Fully Qualified Name Beginning
{node["definition"]["fq_name"]}
# Fully Qualified Name End

# File Paths Beginning
{get_python_files(root_directory)}
# File Paths End

"""
    
    first_user_message+=f"""# Predefined Tools List Beginning
"""
    for i, predefined_tool in enumerate(predefined_tools):

        first_user_message+=f"""{i+1}. Tool Name: {predefined_tool["name"]}
Tool Category: {predefined_tool["category"]}
Tool Imports: {predefined_tool["imports"]}

"""

    first_user_message+=f"""# Predefined Tools List End
"""
    
    first_user_message+=f"""# Custom Tools List Beginning
"""
    for i, custom_tool in enumerate(custom_tools):

        first_user_message+=f"""{i+1}. Tool Name: {custom_tool["name"]}
Tool Category: CUSTOM
Tool Fully Qualified Name: {custom_tool["fully_qualified_name"]}
Tool Fully File Path: {custom_tool["filepath"]}

"""

    first_user_message+=f"""# Custom Tools List End
"""

    messages= [
        {"role":"system", "content": system_prompt},
        {"role": "user", "content": first_user_message}
    ]

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages= messages,
        response_format={"type":"json_object"}
    )

    response_json = json.loads(response.choices[0].message.content)

    print(response_json)

    need_more_information = response_json["need_more_information"]

    while need_more_information:
        new_user_message = ""
        for file_path in response_json["file_paths"]:

            with open(file_path, "r") as f:
                code = f.read()
                new_user_message+=f"""
# Code Beginning
{code}
# Code End

# Code File Path Beginning
{file_path}
# Code File Path End

"""
        
        messages.append({
            "role":"assistant",
            "content": response.choices[0].message.content
        })
        messages.append({
            "role": "user",
            "content": new_user_message
        })

        response = client.chat.completions.create(
            model = "gpt-4o",
            messages= messages,
            response_format={"type":"json_object"}
        )

        response_json = json.loads(response.choices[0].message.content)

        print(response_json)

        need_more_information = response_json["need_more_information"]

In [479]:
for graph in results:

    print(f"Processing nodes from {graph["graph_name"]}")

    for node in graph["graph_info"]["nodes"]:
        print(f"Node: {node["name"]}")
        parse_tool_access(node, graph["graph_file_path"], root_directory, possible_tools, custom_tools)

Processing nodes from workflow
Node: input_node
{'notes': "I need to see the content of the file './Gladiator2/nodes/nodes.py' in order to trace the execution of the 'input_node'.", 'need_more_information': True, 'file_paths': ['./Gladiator2/nodes/nodes.py']}
{'notes': "The 'input_node' function uses 'legal_docs_vector_store_class' and 'Command' components. I need to see the content of the file './Gladiator2/Tools/tools.py' to understand 'legal_docs_vector_store_class', and './Gladiator2/langgraph/types.py' to understand 'Command'.", 'need_more_information': True, 'file_paths': ['./Gladiator2/Tools/tools.py', './Gladiator2/langgraph/types.py']}


FileNotFoundError: [Errno 2] No such file or directory: './Gladiator2/langgraph/types.py'